In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score

import os
import re
import gc
import warnings
import pickle
from tqdm import tqdm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
companies = ['AFLT', 'VTBR', 'GAZP', 'GMKN', 'LKOH', 'ROSN', 'NVTK', 'SBER', 'TCSG', 'YNDX']

In [ ]:
data = pd.read_parquet('/content/drive/MyDrive/Нейросети для исследований научка/data.parquet')
tg_train = data[data.date < '2023-01-01'].copy()
tg_test = data[(data.date >= '2023-01-01')].copy()

In [ ]:
def run_calculations(n_cls, tfidf, embs, ts, classifier, param_grid, files_name, tfidf_params={'sublinear_tf': True, 'min_df': 5, 'ngram_range': (1, 3), 'max_features': 1000},
                    cv=TimeSeriesSplit(), root='/content/drive/MyDrive/Нейросети для исследований/tg_loggs/'
                    df=None, num_cols=['5_min', '10_min', '15_min', '30_min', '1_hour', '1_day'], companies=companies):

  features_train = None
  features_test = None

  target_cols = [col + f'_{n_cls}_cls' for col in num_cols]

  if not df:
    df = pd.DataFrame(index=['acc_test', 'pre_test', 'rec_test', 'auc_test', 'f1_test', 'kappa_test'])

  if embs:
    emb_train = emb_data[:len(tg_train), :]
    emb_test = emb_data[len(tg_train):, :]

  if tfidf:
    tfidf = TfidfVectorizer(**tfidf_params)
    tfidf.fit(tg_train.message_lemm.values)

  for company in companies:
    print(company)
    os.makedirs(f'{root}{company}', exist_ok=True)

    comp_train = tg_train[tg_train['company'] == company].copy()
    comp_test = tg_test[tg_test['company'] == company].copy()

    if tfidf:
      features_train = tfidf.transform(comp_train['message_lemm'].values)
      features_test = tfidf.transform(comp_test['message_lemm'].values)
      features_train = np.asarray(features_train.todense())
      features_test = np.asarray(features_test.todense())

    if embs:
      emb_features_train = emb_train[tg_train['company'] == company]
      emb_features_test = emb_test[tg_test['company'] == company]

      if features_train:
        features_train = np.concatenate([features_train, emb_features_train], axis=1)
        features_test = np.concatenate([features_test, emb_features_test], axis=1)
      else:
        features_train = emb_features_train.copy()
        features_test = emb_features_test.copy()

      del emb_features_train
      del emb_features_test
      gc.collect()

    for t, n in zip(target_cols, num_cols):
      y_train = comp_train[t].values.astype(int)
      y_test = comp_test[t].values.astype(int)

      if ts:
        float_cols = sorted([col for col in tg_train.columns if re.search(f'^{n}_lag', col)])
        num_features_train = comp_train[float_cols]
        num_features_test = comp_test[float_cols]

        if features_train:
          features_train = np.concatenate([features_train, emb_features_train], axis=1)
          features_test = np.concatenate([features_test, emb_features_test], axis=1)
        else:
          features_train = num_features_train.copy()
          features_test = num_features_test.copy()

        del num_features_train
        del num_features_test
        gc.collect()

      gscv = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=cv, n_jobs=1, scoring='accuracy', verbose=2)
      gscv.fit(features_train, y_train)

      preds_test = gscv.predict(features_test)
      preds_proba_test = gscv.predict_proba(features_test)

      acc_test = accuracy_score(y_test, preds_test)
      kappa_test = cohen_kappa_score(y_test, preds_test)

      if n_cls == 3:
        pre_test = precision_score(y_test, preds_test, average='weighted')
        rec_test = recall_score(y_test, preds_test, average='weighted')
        auc_test = roc_auc_score(y_test, preds_proba_test, average='weighted', multi_class='ovo')
        f1_test = f1_score(y_test, preds_test, average='weighted')
      else:
        pre_test = precision_score(y_test, preds_test)
        rec_test = recall_score(y_test, preds_test)
        auc_test = roc_auc_score(y_test, preds_proba_test[:,1])
        f1_test = f1_score(y_test, preds_test)

      df[f'{company}_{t}'] = [acc_test, pre_test, rec_test, auc_test, f1_test, kappa_test]

      filename = f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{company}/{t}_{n_cls}_cls_{files_name}.sav'
      pickle.dump(gscv, open(filename, 'wb'))

      df.to_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_{n_cls}_cls_{files_name}.csv')

## Только временной ряд

### 3 класса

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'forest_num'
n_cls = 3
run_calculations(n_cls=n_cls, tfidf=False, embs=False, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = LGBMClassifier(random_state=123, n_jobs=-1, class_weight='balanced',  n_estimators=200)
files_name = 'boost_num'
n_cls = 3
run_calculations(n_cls=n_cls, tfidf=False, embs=False, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

### 2 класса

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'forest_num'
n_cls = 2
run_calculations(n_cls=n_cls, tfidf=False, embs=False, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = LGBMClassifier(random_state=123, n_jobs=-1, class_weight='balanced',  n_estimators=200)
files_name = 'boost_num'
n_cls = 2
run_calculations(n_cls=n_cls, tfidf=False, embs=False, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

## Только tf-idf

### 3 класса

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'forest_tfidf'
n_cls = 3
run_calculations(n_cls=n_cls, tfidf=True, embs=False, ts=False, classifier=clf, param_grid=param_grid, files_name=files_name)

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = LGBMClassifier(random_state=123, n_jobs=-1, class_weight='balanced',  n_estimators=200)
files_name = 'boost_tfidf'
n_cls = 3
run_calculations(n_cls=n_cls, tfidf=True, embs=False, ts=False, classifier=clf, param_grid=param_grid, files_name=files_name)

### 2 класса

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'forest_tfidf'
n_cls = 2
run_calculations(n_cls=n_cls, tfidf=True, embs=False, ts=False, classifier=clf, param_grid=param_grid, files_name=files_name)

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = LGBMClassifier(random_state=123, n_jobs=-1, class_weight='balanced',  n_estimators=200)
files_name = 'boost_tfidf'
n_cls = 2
run_calculations(n_cls=n_cls, tfidf=True, embs=False, ts=False, classifier=clf, param_grid=param_grid, files_name=files_name)

## Временной ряд + TF-IDF

### Три класса

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'forest'
n_cls = 3
run_calculations(n_cls=n_cls, tfidf=True, embs=False, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'boosting'
n_cls = 3
run_calculations(n_cls=n_cls, tfidf=True, embs=False, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

### Два класса

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'forest'
n_cls = 2
run_calculations(n_cls=n_cls, tfidf=True, embs=False, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'boosting'
n_cls = 2
run_calculations(n_cls=n_cls, tfidf=True, embs=False, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

## Временной ряд + Берт

In [ ]:
emb_data = np.load('/content/drive/MyDrive/Нейросети для исследований научка/bert_embs.npy')

### Три класса

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'forest_embs'
n_cls = 3
run_calculations(n_cls=n_cls, tfidf=False, embs=True, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'boost_embs'
n_cls = 3
run_calculations(n_cls=n_cls, tfidf=False, embs=True, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

### Два класса

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'forest_embs'
n_cls = 2
run_calculations(n_cls=n_cls, tfidf=False, embs=True, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'boost_embs'
n_cls = 2
run_calculations(n_cls=n_cls, tfidf=False, embs=True, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

##Все вместе

In [ ]:
embs = np.load('/content/drive/MyDrive/Нейросети для исследований научка/bert_embs.npy')

### 3 класса

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'forest_all'
n_cls = 3
run_calculations(n_cls=n_cls, tfidf=True, embs=True, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'boost_all'
n_cls = 3
run_calculations(n_cls=n_cls, tfidf=True, embs=True, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

### 2 класса

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'forest_all'
n_cls = 2
run_calculations(n_cls=n_cls, tfidf=True, embs=True, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

In [ ]:
param_grid = {'max_depth': list(range(5, 14))}
clf = RandomForestClassifier(random_state=123, n_estimators=200, criterion='entropy', n_jobs=-1, class_weight='balanced')
files_name = 'boost_all'
n_cls = 2
run_calculations(n_cls=n_cls, tfidf=True, embs=True, ts=True, classifier=clf, param_grid=param_grid, files_name=files_name)

## Тесты на значимость полученных результатов

### Вспомогательные функции

In [ ]:
def sample_test(prob1, y_test, n_cls=2, n_times=1000, zero=False):
  accs = []
  pres = []
  recs = []
  aucs = []
  f1s = []
  kappas = []
  for _ in range(n_times):
    if n_cls == 3:
      if not zero:
        rand_test = np.random.choice([0, 1, 2], size=len(y_test), replace=True, p=prob1)
        for_auc = np.zeros((rand_test.size, 3))
        for_auc[np.arange(rand_test.size), rand_test] = 1
      else:
        rand_test = np.ones(len(y_test))
        for_auc = np.zeros((len(y_test), 3))
        for_auc[:,1] = 1.0

      acc = accuracy_score(y_test, rand_test)
      pre = precision_score(y_test, rand_test, average='weighted')
      rec = recall_score(y_test, rand_test, average='weighted')
      auc = roc_auc_score(y_test, for_auc, average='weighted', multi_class='ovo')
      f1 = f1_score(y_test, rand_test, average='weighted')
      kappa = cohen_kappa_score(y_test, rand_test)
    else:
      rand_test = np.random.choice([0, 1], size=len(y_test), replace=True, p=[1-prob1, prob1])
      acc = accuracy_score(y_test, rand_test)
      pre = precision_score(y_test, rand_test)
      rec = recall_score(y_test, rand_test)
      auc = roc_auc_score(y_test, rand_test)
      f1 = f1_score(y_test, rand_test)
      kappa =cohen_kappa_score(y_test, rand_test)

    accs.append(acc)
    pres.append(pre)
    recs.append(rec)
    aucs.append(auc)
    f1s.append(f1)
    kappas.append(kappa)

  return np.mean(accs) + np.std(accs), np.mean(pres) + np.std(pres), np.mean(recs) + np.std(recs), np.mean(aucs) + np.std(aucs), np.mean(f1s) + np.std(f1s), np.mean(kappas) + np.std(kappas)

In [ ]:
def _color_red_or_green(val):
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color

In [ ]:
def get_effects_bin(df_train, df_test, loggs, companies=companies):
  effects = pd.DataFrame(index=loggs.index)
  num_cols = ['5_min', '10_min', '15_min', '30_min', '1_hour', '1_day']
  target_cols = [col + '_2_cls' for col in num_cols]

  for company in companies:
    comp_train = df_train[df_train[company] == True].copy()
    comp_test = df_test[df_test[company] == True].copy()

    for t in target_cols:
      ba = comp_train[t].mean()
      te = loggs[f'{company}_{t}']
      best_acc = sample_test(ba, comp_test[t].values.astype(int), n_times=1000)
      effects[f'{company}_{t}'] = np.round((te-best_acc)*100, 2)
  return effects

In [ ]:
def get_effects_neutral(df_train, df_test, loggs, companies=companies):
  effects_rand = pd.DataFrame(index=['acc_test', 'pre_test', 'rec_test', 'auc_test', 'f1_test', 'kappa_test'])
  effects_zero = pd.DataFrame(index=['acc_test', 'pre_test', 'rec_test', 'auc_test', 'f1_test', 'kappa_test'])
  num_cols = ['5_min', '10_min', '15_min', '30_min', '1_hour', '1_day']
  target_cols = [col + '_3_cls' for col in num_cols]

  for company in companies:
    comp_train = df_train[df_train[company] == True].copy()
    comp_test = df_test[df_test[company] == True].copy()

    for t in target_cols:
      neg_p = (comp_train[t] == 0).sum() / len(comp_train[t])
      neu_p = (comp_train[t] == 1).sum() / len(comp_train[t])
      pos_p = (comp_train[t] == 2).sum() / len(comp_train[t])

      te = loggs[f'{company}_{t}']

      rand = sample_test([neg_p, neu_p, pos_p], comp_test[t].values, n_times=1000, n_cls=3)
      zero = sample_test([neg_p, neu_p, pos_p], comp_test[t].values, n_times=1, n_cls=3, zero=True)

      effects_rand[f'{company}_{t}'] = np.round((te-rand)*100, 2)
      effects_zero[f'{company}_{t}'] = np.round((te-zero)*100, 2)

  return effects_rand, effects_zero

### Только tf-idf

#### Лес 2 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_2_cls_forest_tfidf.csv', index_col=0)

In [ ]:
rf_2_cls_tfidf = get_effects_bin(tg_train, tg_test, loggs)

In [ ]:
rf_2_cls_tfidf.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_2_cls,AFLT_10_min_2_cls,AFLT_15_min_2_cls,AFLT_30_min_2_cls,AFLT_1_hour_2_cls,AFLT_1_day_2_cls,VTBR_5_min_2_cls,VTBR_10_min_2_cls,VTBR_15_min_2_cls,VTBR_30_min_2_cls,VTBR_1_hour_2_cls,VTBR_1_day_2_cls,GAZP_5_min_2_cls,GAZP_10_min_2_cls,GAZP_15_min_2_cls,GAZP_30_min_2_cls,GAZP_1_hour_2_cls,GAZP_1_day_2_cls,GMKN_5_min_2_cls,GMKN_10_min_2_cls,GMKN_15_min_2_cls,GMKN_30_min_2_cls,GMKN_1_hour_2_cls,GMKN_1_day_2_cls,LKOH_5_min_2_cls,LKOH_10_min_2_cls,LKOH_15_min_2_cls,LKOH_30_min_2_cls,LKOH_1_hour_2_cls,LKOH_1_day_2_cls,ROSN_5_min_2_cls,ROSN_10_min_2_cls,ROSN_15_min_2_cls,ROSN_30_min_2_cls,ROSN_1_hour_2_cls,ROSN_1_day_2_cls,NVTK_5_min_2_cls,NVTK_10_min_2_cls,NVTK_15_min_2_cls,NVTK_30_min_2_cls,NVTK_1_hour_2_cls,NVTK_1_day_2_cls,SBER_5_min_2_cls,SBER_10_min_2_cls,SBER_15_min_2_cls,SBER_30_min_2_cls,SBER_1_hour_2_cls,SBER_1_day_2_cls,TCSG_5_min_2_cls,TCSG_10_min_2_cls,TCSG_15_min_2_cls,TCSG_30_min_2_cls,TCSG_1_hour_2_cls,TCSG_1_day_2_cls,YNDX_5_min_2_cls,YNDX_10_min_2_cls,YNDX_15_min_2_cls,YNDX_30_min_2_cls,YNDX_1_hour_2_cls,YNDX_1_day_2_cls
acc_test,1.31,-3.85,-5.00,1.04,-5.53,0.46,-0.13,-0.80,-0.64,-1.67,-2.59,-3.61,-0.55,1.10,-0.30,-1.27,-1.00,-2.42,-2.28,-1.35,-1.52,3.08,-3.46,0.50,2.43,-0.63,-1.10,-0.85,-2.93,-2.90,-1.39,2.01,1.22,-1.68,-1.25,-2.77,-0.24,-3.46,-3.48,-1.17,0.66,-2.08,-1.88,-1.54,-1.29,-0.76,-3.27,-1.35,-1.90,-5.87,-7.42,-0.24,-1.57,-1.64,-0.84,-0.44,2.06,-2.47,-1.86,-0.94
pre_test,4.12,-2.02,-3.92,2.58,-2.93,-1.42,0.03,-0.82,0.08,-1.61,-2.55,-3.90,-0.62,1.17,-0.05,-1.23,-0.99,-2.01,-2.23,-1.35,-1.21,3.20,-3.26,0.53,1.42,0.18,0.11,-0.84,-2.80,-3.78,-1.54,3.63,1.07,-1.71,-1.19,-3.50,-0.92,-3.15,-3.68,-0.93,1.26,-2.26,-1.34,-1.58,-1.12,-0.80,-3.07,-1.49,-1.70,-5.06,-6.32,-0.33,-1.39,-2.45,-0.64,-0.93,1.16,-2.77,-1.75,-0.75
rec_test,-10.78,-15.92,-20.99,-8.67,-23.75,-19.06,-2.39,-0.60,-9.47,-0.33,-8.27,-10.44,1.94,-1.11,-5.69,-8.67,2.57,1.16,-1.45,-5.39,-8.69,-3.54,-8.15,0.98,9.45,-11.60,-16.91,-17.37,-2.47,-13.17,5.73,-11.05,1.43,-4.71,-5.92,-10.89,3.70,-7.64,1.27,4.24,-6.58,-5.89,-10.05,-0.33,-3.36,-3.32,-1.54,-4.70,0.64,-0.48,-1.57,8.98,1.96,-11.91,-2.98,8.68,7.23,-7.80,-2.51,3.60
auc_test,4.02,-2.94,-5.63,1.77,-3.08,-2.78,0.67,-0.64,2.33,-0.79,-3.32,-4.92,0.31,1.76,0.61,-0.46,-0.55,-2.70,-1.44,-1.62,-1.58,2.70,-5.49,-0.09,1.57,-0.92,-0.63,-1.19,-2.90,-2.99,-0.10,4.33,2.31,0.05,-2.44,-3.13,-1.92,-1.17,-3.37,-1.40,3.44,-2.17,-0.98,-0.25,-1.21,-0.49,-2.15,-1.55,-1.38,-5.39,-7.62,-0.56,-0.75,-3.70,1.19,0.57,3.32,-1.24,-1.66,-0.60
f1_test,-4.69,-10.42,-14.46,-3.61,-17.05,-10.80,-1.13,-0.59,-5.18,-0.89,-5.50,-7.42,0.67,0.10,-2.90,-5.11,0.75,-0.57,-1.67,-3.22,-5.19,0.06,-5.70,0.88,5.30,-6.25,-9.89,-10.16,-2.51,-8.31,1.90,-4.51,1.40,-3.03,-3.43,-7.04,1.58,-5.41,-1.02,1.50,-2.84,-3.83,-6.16,-0.84,-2.18,-1.92,-2.24,-2.95,-0.53,-2.94,-4.16,3.88,0.20,-7.66,-1.67,3.48,4.15,-5.01,-1.97,1.29
kappa_test,5.10,-4.26,-6.83,3.40,-5.30,-3.18,-0.02,-1.68,-0.31,-3.35,-4.85,-6.91,-1.29,2.30,-0.27,-2.38,-2.08,-4.24,-4.64,-2.74,-2.55,5.35,-6.38,1.17,3.96,-0.35,-0.80,-2.11,-5.85,-6.96,-3.28,4.93,2.33,-3.45,-2.48,-6.47,-1.79,-6.28,-8.03,-1.70,1.77,-4.51,-2.67,-3.33,-2.33,-1.65,-6.49,-2.95,-3.56,-11.34,-14.35,-0.26,-2.92,-4.20,-1.44,-1.84,3.19,-5.49,-3.72,-1.43


In [ ]:
rf_2_cls_tfidf.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_2_cls_tfidf.csv')

#### Бустинг 2 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_2_cls_boost_tfidf.csv', index_col=0)

In [ ]:
boost_2_cls_tfidf = get_effects_bin(tg_train, tg_test, loggs)

In [ ]:
boost_2_cls_tfidf.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_2_cls,AFLT_10_min_2_cls,AFLT_15_min_2_cls,AFLT_30_min_2_cls,AFLT_1_hour_2_cls,AFLT_1_day_2_cls,VTBR_5_min_2_cls,VTBR_10_min_2_cls,VTBR_15_min_2_cls,VTBR_30_min_2_cls,VTBR_1_hour_2_cls,VTBR_1_day_2_cls,GAZP_5_min_2_cls,GAZP_10_min_2_cls,GAZP_15_min_2_cls,GAZP_30_min_2_cls,GAZP_1_hour_2_cls,GAZP_1_day_2_cls,GMKN_5_min_2_cls,GMKN_10_min_2_cls,GMKN_15_min_2_cls,GMKN_30_min_2_cls,GMKN_1_hour_2_cls,GMKN_1_day_2_cls,LKOH_5_min_2_cls,LKOH_10_min_2_cls,LKOH_15_min_2_cls,LKOH_30_min_2_cls,LKOH_1_hour_2_cls,LKOH_1_day_2_cls,ROSN_5_min_2_cls,ROSN_10_min_2_cls,ROSN_15_min_2_cls,ROSN_30_min_2_cls,ROSN_1_hour_2_cls,ROSN_1_day_2_cls,NVTK_5_min_2_cls,NVTK_10_min_2_cls,NVTK_15_min_2_cls,NVTK_30_min_2_cls,NVTK_1_hour_2_cls,NVTK_1_day_2_cls,SBER_5_min_2_cls,SBER_10_min_2_cls,SBER_15_min_2_cls,SBER_30_min_2_cls,SBER_1_hour_2_cls,SBER_1_day_2_cls,TCSG_5_min_2_cls,TCSG_10_min_2_cls,TCSG_15_min_2_cls,TCSG_30_min_2_cls,TCSG_1_hour_2_cls,TCSG_1_day_2_cls,YNDX_5_min_2_cls,YNDX_10_min_2_cls,YNDX_15_min_2_cls,YNDX_30_min_2_cls,YNDX_1_hour_2_cls,YNDX_1_day_2_cls
acc_test,-3.03,-2.16,-4.06,0.12,-3.80,-2.83,-1.98,-1.36,-0.77,-0.86,-1.03,-3.04,0.91,0.83,0.43,-3.06,-0.51,-0.69,1.11,-1.33,-1.62,-1.62,-0.51,-1.50,0.49,-1.47,-2.78,-0.36,-2.58,-2.09,-2.83,-0.33,-2.64,-3.18,-2.42,-2.03,-2.45,-0.34,-3.21,0.98,1.94,-3.48,0.32,0.28,-0.59,-2.68,-2.32,-4.60,-2.92,-5.91,-1.93,-0.61,1.16,-3.32,-0.92,-1.32,0.95,-0.46,0.26,1.78
pre_test,-3.37,-2.80,-3.91,0.34,-4.17,-3.19,-1.99,-1.40,-0.55,-0.82,-1.02,-3.07,0.58,0.94,0.32,-3.09,-0.50,-0.63,1.04,-1.31,-1.45,-1.75,-0.35,-1.44,0.40,-1.33,-2.49,-0.21,-2.44,-2.85,-2.68,-0.07,-2.42,-3.35,-2.34,-2.12,-2.24,-0.29,-3.35,1.02,2.73,-3.31,0.93,0.24,-0.10,-2.88,-2.30,-4.32,-2.69,-5.27,-1.62,-0.62,1.13,-4.02,-0.73,-1.23,1.06,-0.59,0.28,1.68
rec_test,0.48,2.57,-5.69,-2.50,-0.64,-6.02,-1.33,-0.49,-4.24,-3.65,-1.27,-1.56,5.25,-1.89,1.81,-6.00,-0.70,-0.41,0.69,-4.97,-6.68,-3.93,-2.79,-1.82,0.63,-3.88,-11.74,-11.94,-1.06,-12.50,-7.60,-4.77,-5.75,-8.09,-2.46,-3.91,-3.55,-1.38,0.06,7.26,-5.35,-3.13,-5.65,0.07,-6.03,-6.89,-4.43,-2.69,-1.76,-2.46,4.10,4.16,-0.15,-8.40,-2.89,-2.06,-1.08,-4.30,-2.27,6.36
auc_test,-2.24,-1.28,-5.23,-0.80,-4.06,-1.37,-1.80,-1.53,-0.06,0.02,-0.35,-2.64,0.33,0.44,0.19,-2.64,0.04,-0.66,0.54,-1.77,-1.40,-0.73,-0.65,0.20,0.38,-1.47,-2.12,-1.26,-2.67,-3.46,-1.45,-0.97,-2.27,-1.77,-3.85,-2.62,-2.00,-1.47,-2.43,3.04,3.54,0.33,-0.74,1.34,-0.30,-2.74,-2.64,-6.56,-3.27,-5.00,-2.96,0.30,0.69,-4.65,1.06,-0.19,2.71,0.77,-0.11,1.64
f1_test,-1.14,0.22,-4.64,-0.95,-2.04,-4.29,-1.53,-0.82,-2.38,-2.10,-1.04,-2.19,2.85,-0.41,1.11,-4.49,-0.52,-0.47,1.06,-2.99,-4.11,-2.62,-1.45,-1.47,0.64,-2.50,-7.45,-6.41,-1.66,-7.51,-5.05,-2.33,-4.02,-5.57,-2.26,-2.83,-2.78,-0.69,-1.46,3.91,-1.52,-3.08,-2.59,0.27,-3.19,-4.75,-3.24,-3.47,-2.16,-3.90,0.99,1.69,0.66,-6.22,-1.67,-1.50,0.11,-2.19,-0.80,3.87
kappa_test,-7.10,-5.95,-7.91,0.51,-8.77,-6.09,-4.11,-2.87,-1.19,-1.71,-2.07,-6.17,1.51,1.80,0.76,-6.05,-1.02,-1.28,2.19,-2.69,-2.92,-3.46,-0.82,-2.92,0.90,-2.77,-4.76,-0.99,-5.14,-5.36,-5.39,-0.36,-4.93,-6.52,-4.84,-4.20,-4.72,-0.62,-7.17,2.66,4.33,-6.79,1.40,0.52,-0.44,-5.59,-4.67,-8.95,-5.69,-11.55,-3.43,-1.10,2.28,-7.09,-1.64,-2.64,2.15,-1.29,0.49,3.97


In [ ]:
boost_2_cls_tfidf.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_2_cls_tfidf.csv')

#### Лес 3 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_3_cls_forest_tfidf.csv', index_col=0)

In [ ]:
warnings.simplefilter('ignore')
rf_3_cls_rand_tfidf, rf_3_cls_zero_tfidf = get_effects_neutral(tg_train, tg_test, loggs)
warnings.simplefilter('default')

In [ ]:
rf_3_cls_rand_tfidf.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,12.51,-1.17,-5.73,-0.68,3.17,4.87,8.97,-5.50,0.31,7.41,9.34,1.54,0.02,-7.71,-6.66,-0.22,7.63,-3.36,9.06,4.04,4.89,3.94,4.69,7.83,-3.98,-9.17,-4.70,3.07,12.33,-1.83,-1.50,-7.18,-6.68,2.53,11.04,0.95,-5.50,-5.66,-1.53,4.00,12.78,1.44,9.92,-1.90,1.76,4.14,4.09,3.35,4.87,10.36,7.70,7.04,9.51,10.48,7.96,7.82,5.93,7.58,4.78,5.94
pre_test,-0.83,0.08,0.31,0.01,-0.85,1.51,-0.91,-0.92,0.02,1.34,1.54,-1.18,-0.64,0.24,-0.10,0.57,0.72,-0.25,-1.82,-0.93,-0.62,0.00,-0.34,0.61,-0.55,-1.88,-0.35,0.36,-0.65,-2.34,-2.58,-1.47,-1.19,0.39,1.06,-1.71,-1.62,0.86,2.18,2.96,-1.75,-1.09,-1.08,0.18,-0.58,-0.31,0.32,0.59,-0.06,0.57,-2.20,-1.08,1.44,0.79,-0.69,0.08,0.82,0.81,2.47,-0.16
rec_test,12.51,-1.17,-5.73,-0.68,3.17,4.87,8.97,-5.50,0.31,7.41,9.34,1.54,0.02,-7.71,-6.66,-0.22,7.63,-3.36,9.06,4.04,4.89,3.94,4.69,7.83,-3.98,-9.17,-4.70,3.07,12.33,-1.83,-1.50,-7.18,-6.68,2.53,11.04,0.95,-5.50,-5.66,-1.53,4.00,12.78,1.44,9.92,-1.90,1.76,4.14,4.09,3.35,4.87,10.36,7.70,7.04,9.51,10.48,7.96,7.82,5.93,7.58,4.78,5.94
auc_test,-0.37,2.21,-0.52,0.26,3.55,0.57,-0.94,-0.95,0.53,3.75,1.16,-1.23,-0.94,-1.15,-0.74,0.41,1.98,0.15,-1.92,1.25,0.70,-0.71,2.17,4.44,-1.61,-2.30,0.25,-1.18,-0.73,-1.49,-1.73,-0.99,-1.68,0.56,3.23,-2.25,-0.47,3.52,0.30,1.91,1.94,-0.05,-2.08,0.05,-0.93,-0.11,1.92,0.97,0.15,1.72,-1.91,-3.09,3.03,-4.55,-2.30,0.87,2.35,1.96,2.00,1.12
f1_test,3.93,-1.15,-4.19,-0.47,0.59,1.88,3.75,-3.77,0.21,3.68,4.48,-0.12,-0.36,-4.77,-4.18,0.13,3.46,-1.95,2.84,1.21,1.80,2.11,2.18,3.07,-2.47,-7.45,-3.85,1.68,4.50,-2.25,-2.02,-5.31,-5.23,1.47,4.47,-0.32,-4.40,-3.97,-0.13,3.13,3.09,0.34,3.52,-1.02,0.56,1.92,1.95,2.01,2.06,2.35,0.33,1.02,3.64,3.49,3.05,2.78,2.94,3.80,3.65,2.34
kappa_test,-3.13,-0.37,0.77,-0.11,-1.23,1.93,-1.12,-2.27,0.22,2.37,4.76,-1.46,-1.28,0.02,-0.20,1.06,1.34,-0.59,-3.91,-1.31,-0.47,0.05,0.04,0.21,-1.69,-2.30,0.10,1.56,-0.86,-5.94,-6.22,-1.81,-1.12,1.13,1.93,-3.24,-3.30,2.12,4.38,8.09,-3.44,-1.59,-2.59,0.33,-1.09,-0.23,0.93,1.49,1.51,0.01,-2.60,-0.79,1.55,-0.96,-1.25,-0.25,1.64,1.21,6.44,0.33


In [ ]:
rf_3_cls_zero_tfidf.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,-5.18,-17.73,-23.31,-18.73,-7.97,-6.57,-10.21,-23.98,-15.75,-9.06,-6.81,-13.61,-19.79,-24.10,-23.82,-18.39,-8.28,-19.84,-9.39,-10.97,-10.97,-13.80,-12.22,-7.13,-20.46,-25.37,-20.37,-14.05,-4.48,-17.47,-21.96,-22.40,-23.18,-13.63,-4.25,-14.41,-22.54,-21.16,-16.83,-12.50,-3.35,-12.20,-8.55,-18.13,-13.91,-13.31,-10.50,-13.69,-11.00,-3.61,-5.50,-6.65,-4.76,-6.40,-9.30,-6.96,-9.52,-9.16,-10.62,-10.33
pre_test,5.91,6.80,6.39,4.66,6.85,11.80,5.20,4.72,6.08,6.81,6.37,4.91,4.88,6.22,5.30,5.12,5.46,5.01,4.76,6.96,6.18,5.51,4.59,7.17,6.91,4.92,6.58,6.42,4.05,3.67,2.98,5.52,4.79,6.66,6.96,4.90,5.34,7.71,9.11,9.50,4.12,7.45,5.11,6.19,5.59,4.39,6.09,5.79,7.49,8.69,5.87,6.16,7.26,6.82,5.32,6.37,6.65,5.89,7.79,6.36
rec_test,-5.18,-17.73,-23.31,-18.73,-7.97,-6.57,-10.21,-23.98,-15.75,-9.06,-6.81,-13.61,-19.79,-24.10,-23.82,-18.39,-8.28,-19.84,-9.39,-10.97,-10.97,-13.80,-12.22,-7.13,-20.46,-25.37,-20.37,-14.05,-4.48,-17.47,-21.96,-22.40,-23.18,-13.63,-4.25,-14.41,-22.54,-21.16,-16.83,-12.50,-3.35,-12.20,-8.55,-18.13,-13.91,-13.31,-10.50,-13.69,-11.00,-3.61,-5.50,-6.65,-4.76,-6.40,-9.30,-6.96,-9.52,-9.16,-10.62,-10.33
auc_test,1.29,3.87,1.31,2.08,5.05,2.11,-0.03,-0.01,1.38,4.62,2.03,-0.37,-0.35,-0.55,-0.16,1.03,2.59,0.76,-0.66,2.54,1.85,0.61,3.42,5.59,-0.48,-1.22,1.41,-0.07,0.38,-0.25,-0.51,0.10,-0.63,1.65,4.30,-1.13,0.69,4.70,1.49,3.08,3.24,1.11,-1.20,0.93,-0.07,0.81,2.72,1.82,1.18,2.77,-0.94,-2.10,4.06,-3.50,-1.23,1.91,3.31,2.95,3.06,2.16
f1_test,0.27,-4.17,-8.42,-6.10,0.80,3.60,-0.92,-8.44,-2.76,0.01,0.45,-2.59,-5.74,-7.84,-8.15,-5.12,-0.43,-5.68,-1.15,0.06,-0.58,-2.45,-2.34,0.94,-4.64,-10.03,-6.07,-1.99,-0.08,-5.21,-8.04,-7.17,-8.57,-1.46,1.61,-2.73,-7.25,-6.19,-2.23,0.17,-0.23,-0.01,-0.66,-4.11,-2.13,-2.93,-0.57,-2.16,0.33,1.76,0.07,-0.18,1.28,-0.20,-0.65,0.55,0.05,-0.38,0.25,-0.41
kappa_test,0.06,2.88,4.20,3.27,2.06,5.12,0.60,-0.47,1.94,4.10,6.50,0.32,-0.15,1.23,0.96,2.26,2.58,0.63,-1.52,1.32,1.85,2.58,2.47,2.55,0.56,-0.13,2.46,3.74,1.30,-3.48,-4.00,0.42,0.97,3.33,4.12,-1.03,-1.01,4.51,6.81,10.42,-0.85,0.80,-0.95,2.10,0.66,1.55,2.59,3.18,3.58,2.15,-0.55,1.27,3.70,1.07,0.84,1.90,3.59,3.14,8.56,2.42


In [ ]:
rf_3_cls_rand_tfidf.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_3_cls_rand_tfidf.csv')
rf_3_cls_zero_tfidf.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_3_cls_zero_tfidf.csv')

#### Бустинг 3 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_3_cls_boost_tfidf.csv', index_col=0)

In [ ]:
warnings.simplefilter('ignore')
boost_3_cls_rand_tfidf, boost_3_cls_zero_tfidf = get_effects_neutral(tg_train, tg_test, loggs)
warnings.simplefilter('default')

In [ ]:
boost_3_cls_rand_tfidf.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,2.46,1.35,-3.24,-1.96,0.77,-1.75,-0.86,-6.33,-5.27,2.04,-1.30,-3.26,-5.26,-6.11,-6.15,-2.88,-3.46,-1.93,0.80,2.19,3.79,4.59,4.21,4.00,0.26,-0.88,-3.49,-1.90,3.32,-1.10,-0.54,-3.51,-2.78,0.43,5.55,-2.28,-1.35,2.00,-3.83,1.80,3.44,3.42,-1.73,-7.64,-3.86,-3.53,-2.02,-3.45,4.09,5.11,5.03,4.43,4.86,6.49,-0.40,4.62,2.05,-0.22,1.27,2.80
pre_test,-1.52,-0.83,-0.86,1.25,-0.64,-4.27,-0.87,-2.55,-1.76,1.37,-0.18,-1.05,-0.70,-0.72,-0.16,0.70,0.60,0.67,-2.13,1.13,-0.56,-1.32,-0.75,-0.61,-0.99,-1.10,-1.77,-1.74,-0.92,-0.79,-1.14,-2.55,-0.66,-0.47,1.04,-2.50,-1.67,0.26,-1.51,0.16,-2.03,0.13,-0.48,-2.48,0.00,-1.42,0.65,-0.33,0.94,-0.59,-0.57,-1.12,-1.04,-0.40,-1.33,0.42,-0.46,-0.12,0.44,0.42
rec_test,2.46,1.35,-3.24,-1.96,0.77,-1.75,-0.86,-6.33,-5.27,2.04,-1.30,-3.26,-5.26,-6.11,-6.15,-2.88,-3.46,-1.93,0.80,2.19,3.79,4.59,4.21,4.00,0.26,-0.88,-3.49,-1.90,3.32,-1.10,-0.54,-3.51,-2.78,0.43,5.55,-2.28,-1.35,2.00,-3.83,1.80,3.44,3.42,-1.73,-7.64,-3.86,-3.53,-2.02,-3.45,4.09,5.11,5.03,4.43,4.86,6.49,-0.40,4.62,2.05,-0.22,1.27,2.80
auc_test,-2.80,2.05,0.46,-0.94,-1.46,-0.82,-0.53,-3.04,-0.98,2.40,0.79,-0.26,-0.76,0.04,0.16,0.29,2.14,1.52,-3.86,1.05,-1.61,0.58,-0.84,1.49,-0.04,-2.83,-2.41,-1.35,-0.50,-0.30,0.02,-1.42,-0.85,0.02,1.01,-3.32,-1.23,4.45,-0.40,1.23,0.61,-0.28,-1.83,-2.63,0.92,-0.94,1.36,0.57,1.02,0.65,-2.28,-4.23,2.15,0.14,-1.14,-0.96,-0.98,-0.41,-0.05,1.01
f1_test,0.62,0.39,-2.24,-0.53,-0.14,-3.11,-0.83,-4.84,-3.65,1.76,-0.72,-2.14,-3.74,-3.87,-3.78,-1.43,-1.63,-0.73,-0.47,1.70,1.48,1.70,1.69,1.27,-0.28,-0.92,-2.78,-1.77,1.23,-0.92,-0.80,-3.20,-1.99,-0.11,3.05,-2.27,-1.48,1.10,-2.64,1.07,0.55,1.58,-1.19,-5.54,-2.15,-2.56,-0.62,-2.07,2.44,1.29,1.12,0.92,1.32,2.77,-0.78,2.14,0.73,-0.13,0.92,1.62
kappa_test,-2.83,-1.65,-1.65,1.95,-1.05,-6.20,-1.78,-4.81,-3.73,2.90,-0.43,-2.07,-1.40,-1.42,-0.64,1.41,1.25,1.32,-4.13,2.28,-0.53,-2.45,-0.95,-1.55,-1.86,-2.14,-3.20,-3.51,-1.85,-1.79,-2.11,-4.34,-0.91,-0.49,2.34,-4.98,-3.02,0.75,-3.19,0.35,-3.88,0.44,-1.09,-4.44,-0.04,-3.02,1.17,-0.83,2.00,-1.28,-0.71,-1.84,-1.77,-0.82,-2.73,0.37,-0.98,0.03,0.89,0.85


In [ ]:
boost_3_cls_zero_tfidf.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,-14.94,-15.14,-20.92,-19.92,-10.36,-13.15,-20.03,-24.92,-21.41,-14.43,-17.40,-18.44,-25.08,-22.55,-23.27,-21.07,-19.37,-18.42,-17.53,-12.90,-11.99,-13.12,-12.67,-10.86,-16.24,-17.03,-19.23,-18.88,-13.52,-16.94,-21.09,-18.75,-19.27,-15.62,-9.81,-17.62,-18.31,-13.48,-19.19,-14.67,-12.70,-10.24,-20.13,-23.86,-19.53,-21.00,-16.61,-20.51,-11.74,-8.78,-8.21,-9.20,-9.44,-10.26,-17.66,-10.18,-13.26,-17.00,-14.14,-13.55
pre_test,5.42,6.04,5.17,5.91,7.08,6.04,5.31,3.02,4.30,6.82,4.63,4.99,4.83,5.25,5.30,5.27,5.34,5.93,4.51,8.94,6.33,4.21,4.22,6.09,6.47,5.80,5.05,4.43,3.85,5.02,4.38,4.37,5.33,5.82,6.87,4.14,5.31,7.19,5.36,6.75,3.79,8.63,5.77,3.51,6.10,3.30,6.41,4.87,8.54,7.59,7.52,6.11,4.74,5.74,4.66,6.65,5.45,4.98,5.74,6.88
rec_test,-14.94,-15.14,-20.92,-19.92,-10.36,-13.15,-20.03,-24.92,-21.41,-14.43,-17.40,-18.44,-25.08,-22.55,-23.27,-21.07,-19.37,-18.42,-17.53,-12.90,-11.99,-13.12,-12.67,-10.86,-16.24,-17.03,-19.23,-18.88,-13.52,-16.94,-21.09,-18.75,-19.27,-15.62,-9.81,-17.62,-18.31,-13.48,-19.19,-14.67,-12.70,-10.24,-20.13,-23.86,-19.53,-21.00,-16.61,-20.51,-11.74,-8.78,-8.21,-9.20,-9.44,-10.26,-17.66,-10.18,-13.26,-17.00,-14.14,-13.55
auc_test,-0.92,3.84,2.23,0.96,0.04,0.70,0.46,-2.17,-0.13,3.24,1.63,0.55,-0.14,0.62,0.80,0.94,2.74,2.13,-2.56,2.26,-0.37,1.95,0.47,2.77,1.09,-1.65,-1.36,-0.13,0.68,0.72,1.20,-0.39,0.20,1.13,2.02,-2.20,-0.05,5.71,0.74,2.46,1.87,0.91,-0.92,-1.78,1.72,-0.01,2.16,1.43,2.08,1.74,-1.31,-3.24,3.15,1.30,-0.11,0.02,0.06,0.58,1.00,1.99
f1_test,-2.80,-2.52,-6.54,-6.12,0.09,-1.36,-5.46,-9.59,-6.66,-1.92,-4.73,-4.65,-9.11,-6.97,-7.69,-6.68,-5.51,-4.46,-4.37,0.48,-0.81,-2.84,-2.79,-0.73,-2.46,-3.42,-5.10,-5.30,-3.34,-4.09,-6.90,-5.11,-5.32,-2.97,0.14,-4.65,-4.27,-1.06,-4.82,-1.85,-2.80,1.21,-5.30,-8.64,-4.87,-7.42,-3.15,-6.24,0.76,0.76,0.86,-0.25,-1.06,-0.80,-4.49,-0.12,-2.05,-4.33,-2.49,-1.21
kappa_test,0.75,1.87,1.69,5.43,2.27,-3.01,0.08,-3.15,-2.02,4.59,1.28,-0.40,-0.25,-0.24,0.65,2.67,2.48,2.54,-1.65,4.76,1.98,0.15,1.58,1.08,0.39,0.23,-1.05,-1.10,0.45,0.25,0.05,-2.22,1.19,1.76,4.40,-2.74,-0.70,3.31,-0.90,2.78,-1.38,2.78,0.65,-2.70,1.58,-1.20,2.83,0.85,4.16,0.96,1.35,0.22,0.31,1.43,-0.70,2.42,1.15,1.99,2.98,2.81


In [ ]:
boost_3_cls_rand_tfidf.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_3_cls_rand_tfidf.csv')
boost_3_cls_zero_tfidf.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_3_cls_zero_tfidf.csv')

### Только временной ряд

#### Лес 2 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_2_cls_forest_num.csv', index_col=0)

In [ ]:
rf_2_cls_num = get_effects_bin(tg_train, tg_test, loggs)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
rf_2_cls_num.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_2_cls,AFLT_10_min_2_cls,AFLT_15_min_2_cls,AFLT_30_min_2_cls,AFLT_1_hour_2_cls,AFLT_1_day_2_cls,VTBR_5_min_2_cls,VTBR_10_min_2_cls,VTBR_15_min_2_cls,VTBR_30_min_2_cls,VTBR_1_hour_2_cls,VTBR_1_day_2_cls,GAZP_5_min_2_cls,GAZP_10_min_2_cls,GAZP_15_min_2_cls,GAZP_30_min_2_cls,GAZP_1_hour_2_cls,GAZP_1_day_2_cls,GMKN_5_min_2_cls,GMKN_10_min_2_cls,GMKN_15_min_2_cls,GMKN_30_min_2_cls,GMKN_1_hour_2_cls,GMKN_1_day_2_cls,LKOH_5_min_2_cls,LKOH_10_min_2_cls,LKOH_15_min_2_cls,LKOH_30_min_2_cls,LKOH_1_hour_2_cls,LKOH_1_day_2_cls,ROSN_5_min_2_cls,ROSN_10_min_2_cls,ROSN_15_min_2_cls,ROSN_30_min_2_cls,ROSN_1_hour_2_cls,ROSN_1_day_2_cls,NVTK_5_min_2_cls,NVTK_10_min_2_cls,NVTK_15_min_2_cls,NVTK_30_min_2_cls,NVTK_1_hour_2_cls,NVTK_1_day_2_cls,SBER_5_min_2_cls,SBER_10_min_2_cls,SBER_15_min_2_cls,SBER_30_min_2_cls,SBER_1_hour_2_cls,SBER_1_day_2_cls,TCSG_5_min_2_cls,TCSG_10_min_2_cls,TCSG_15_min_2_cls,TCSG_30_min_2_cls,TCSG_1_hour_2_cls,TCSG_1_day_2_cls,YNDX_5_min_2_cls,YNDX_10_min_2_cls,YNDX_15_min_2_cls,YNDX_30_min_2_cls,YNDX_1_hour_2_cls,YNDX_1_day_2_cls
acc_test,14.85,13.87,8.48,20.52,14.21,23.65,10.56,9.17,9.76,18.14,14.12,19.67,12.82,13.89,13.13,18.03,12.75,18.22,13.13,12.92,6.11,13.94,6.82,15.67,8.32,8.81,4.78,11.47,10.26,12.75,7.97,9.48,6.04,13.09,9.14,13.15,9.24,8.09,11.89,11.55,14.05,15.25,15.47,11.48,10.95,14.84,10.23,19.41,12.55,10.14,8.56,10.90,10.83,12.56,8.51,10.47,6.09,13.94,10.91,15.80
pre_test,12.48,15.96,9.14,25.57,14.02,30.20,11.37,8.25,9.20,18.55,12.80,20.66,12.37,12.52,11.72,20.34,14.79,19.63,12.00,12.93,7.52,17.31,6.61,18.50,8.70,11.65,7.09,13.54,10.86,14.51,8.56,13.51,8.10,16.20,12.71,15.18,8.07,9.00,12.55,14.32,19.08,20.19,14.27,10.30,9.27,15.83,11.57,22.67,10.23,8.64,7.27,13.34,9.39,10.69,9.27,12.92,9.71,15.81,13.57,18.05
rec_test,17.34,6.75,4.39,9.87,11.68,14.44,6.27,10.91,9.99,15.06,16.84,17.51,11.36,15.81,15.32,11.29,4.65,14.13,14.40,11.85,-0.91,2.85,5.71,5.58,4.26,-2.47,-7.27,2.87,5.76,3.77,2.92,-4.79,-3.72,2.31,-5.55,5.22,9.56,2.89,8.06,0.66,1.43,0.42,16.45,12.66,14.11,10.80,2.87,11.69,22.78,16.85,15.65,0.42,16.01,27.04,2.17,-0.18,-8.42,4.91,-2.10,7.92
auc_test,16.13,18.56,13.85,27.76,22.61,30.88,15.94,14.43,15.12,23.26,20.65,23.12,19.15,18.51,17.42,24.22,20.45,24.27,16.24,19.36,12.51,19.34,10.71,20.51,13.83,12.65,10.58,17.37,14.41,18.67,13.23,13.87,12.56,19.11,15.55,19.94,15.05,12.96,14.24,15.63,17.26,19.31,20.25,15.68,16.26,21.06,16.03,23.61,19.91,15.34,13.19,16.52,16.81,22.46,14.81,14.57,13.55,19.66,18.13,21.35
f1_test,15.21,11.00,6.80,16.89,13.00,22.34,8.76,9.69,9.71,16.92,14.89,19.13,11.95,14.18,13.51,15.65,9.49,17.06,13.34,12.55,3.07,9.91,6.31,12.18,6.55,3.91,-0.91,8.07,8.46,9.30,5.82,3.21,1.65,8.90,2.61,10.26,9.01,5.87,10.29,7.23,9.13,9.63,15.49,11.59,11.76,13.45,7.20,17.10,15.67,12.37,11.03,6.58,12.45,18.12,5.73,5.96,-0.64,10.52,5.41,13.04
kappa_test,28.97,28.77,17.40,41.85,28.46,46.28,21.49,18.12,19.43,36.29,28.10,39.36,25.72,27.70,26.15,36.15,25.60,35.95,26.19,25.84,12.60,26.73,13.67,30.24,16.98,18.39,10.49,22.79,20.49,24.65,16.18,19.80,13.08,25.89,18.32,25.56,18.16,16.76,24.15,22.23,28.69,29.25,30.74,22.75,21.45,29.49,20.37,38.38,25.61,20.56,17.48,21.63,21.87,26.18,17.73,21.73,14.23,27.03,21.65,31.11


In [ ]:
rf_2_cls_num.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_2_cls_num.csv')

#### Бустинг 2 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_2_cls_boost_num.csv', index_col=0)

In [ ]:
boost_2_cls_num = get_effects_bin(tg_train, tg_test, loggs)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
boost_2_cls_num.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_2_cls,AFLT_10_min_2_cls,AFLT_15_min_2_cls,AFLT_30_min_2_cls,AFLT_1_hour_2_cls,AFLT_1_day_2_cls,VTBR_5_min_2_cls,VTBR_10_min_2_cls,VTBR_15_min_2_cls,VTBR_30_min_2_cls,VTBR_1_hour_2_cls,VTBR_1_day_2_cls,GAZP_5_min_2_cls,GAZP_10_min_2_cls,GAZP_15_min_2_cls,GAZP_30_min_2_cls,GAZP_1_hour_2_cls,GAZP_1_day_2_cls,GMKN_5_min_2_cls,GMKN_10_min_2_cls,GMKN_15_min_2_cls,GMKN_30_min_2_cls,GMKN_1_hour_2_cls,GMKN_1_day_2_cls,LKOH_5_min_2_cls,LKOH_10_min_2_cls,LKOH_15_min_2_cls,LKOH_30_min_2_cls,LKOH_1_hour_2_cls,LKOH_1_day_2_cls,ROSN_5_min_2_cls,ROSN_10_min_2_cls,ROSN_15_min_2_cls,ROSN_30_min_2_cls,ROSN_1_hour_2_cls,ROSN_1_day_2_cls,NVTK_5_min_2_cls,NVTK_10_min_2_cls,NVTK_15_min_2_cls,NVTK_30_min_2_cls,NVTK_1_hour_2_cls,NVTK_1_day_2_cls,SBER_5_min_2_cls,SBER_10_min_2_cls,SBER_15_min_2_cls,SBER_30_min_2_cls,SBER_1_hour_2_cls,SBER_1_day_2_cls,TCSG_5_min_2_cls,TCSG_10_min_2_cls,TCSG_15_min_2_cls,TCSG_30_min_2_cls,TCSG_1_hour_2_cls,TCSG_1_day_2_cls,YNDX_5_min_2_cls,YNDX_10_min_2_cls,YNDX_15_min_2_cls,YNDX_30_min_2_cls,YNDX_1_hour_2_cls,YNDX_1_day_2_cls
acc_test,6.95,10.70,4.73,17.77,14.57,19.96,9.84,9.65,9.22,16.85,12.81,15.97,14.06,12.80,12.31,16.37,12.64,17.56,12.20,9.16,4.28,12.53,7.74,11.86,8.36,8.05,3.43,11.36,8.86,10.49,9.68,9.64,4.29,12.40,7.14,14.00,7.04,5.96,8.28,9.86,12.29,14.23,12.37,10.89,9.25,14.09,9.76,17.13,11.46,9.31,6.21,5.80,10.28,8.99,6.28,5.22,6.23,11.77,11.08,14.67
pre_test,6.75,9.92,5.48,22.31,12.09,23.78,9.70,9.39,8.94,17.49,12.08,17.11,13.37,12.61,11.96,17.33,13.61,18.62,10.35,9.59,4.41,13.90,7.21,13.20,8.68,8.78,3.94,13.66,9.92,11.40,9.93,12.05,5.12,14.42,8.54,15.04,5.65,6.18,8.84,10.66,15.84,17.05,11.64,10.89,8.66,15.18,10.92,23.01,10.77,9.01,5.87,6.85,9.22,8.98,6.08,7.07,8.18,12.40,12.48,16.55
rec_test,5.59,9.71,0.14,7.28,16.88,13.28,8.39,8.71,8.48,13.16,13.82,13.15,13.14,10.89,10.94,12.43,7.63,14.23,16.62,5.97,2.78,7.05,7.77,4.17,4.59,3.03,-1.26,2.17,1.92,3.74,6.06,-0.47,-1.66,4.17,-1.79,10.29,9.14,3.01,4.67,5.64,1.67,3.86,12.68,8.39,9.02,9.46,2.77,3.19,12.68,8.64,6.61,-2.88,13.30,13.25,3.91,-6.01,-3.72,6.38,2.23,7.36
auc_test,12.41,14.98,9.46,25.02,18.29,25.55,14.52,13.87,13.96,22.90,18.27,21.89,19.29,17.25,15.86,23.06,19.94,23.96,16.21,14.56,6.93,18.39,10.21,17.60,13.08,9.23,7.02,14.44,13.03,16.24,12.72,14.19,9.27,17.55,12.49,18.51,9.25,10.70,12.14,15.18,16.41,17.73,17.68,14.63,13.45,20.00,14.49,22.72,17.31,11.32,10.14,10.16,14.02,14.80,10.70,10.14,11.65,16.09,15.35,17.82
f1_test,6.37,10.04,2.81,14.02,14.88,18.81,9.14,9.16,8.82,15.44,13.06,15.14,13.34,11.83,11.53,14.89,10.61,16.62,13.44,7.94,3.71,10.69,7.66,8.98,6.70,5.93,1.35,7.71,5.98,7.83,8.12,5.37,1.63,9.22,3.24,12.87,7.61,4.68,6.74,8.32,8.05,10.41,12.28,9.72,8.96,12.43,6.85,12.14,11.79,8.99,6.35,1.91,11.19,11.19,5.14,0.09,1.79,9.68,7.42,12.05
kappa_test,13.91,21.26,10.06,36.42,28.10,39.01,19.75,19.32,18.44,33.71,25.54,31.98,28.17,25.68,24.69,32.80,25.34,34.70,24.21,18.29,8.60,24.45,15.38,22.81,17.05,16.42,7.21,22.55,17.67,20.32,19.53,19.88,9.20,24.59,14.30,27.66,13.31,12.21,16.96,19.37,25.10,27.56,24.62,21.97,18.43,27.96,19.43,33.44,22.97,18.58,12.44,11.43,20.67,18.37,12.80,11.36,13.84,22.98,22.05,28.87


In [ ]:
boost_2_cls_num.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_2_cls_num.csv')

#### Лес 3 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_3_cls_forest_num.csv', index_col=0)

In [ ]:
warnings.simplefilter('ignore')
rf_3_cls_rand_num, rf_3_cls_zero_num = get_effects_neutral(tg_train, tg_test, loggs)
warnings.simplefilter('default')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
rf_3_cls_rand_num.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,33.62,29.79,26.06,24.76,27.67,37.22,28.43,30.06,29.83,26.30,25.83,26.15,34.62,32.75,32.01,30.37,30.40,29.42,24.80,25.66,25.99,22.83,25.25,27.39,23.34,18.16,23.86,21.12,21.39,21.44,24.15,22.29,19.81,23.04,25.06,21.87,24.71,21.85,23.27,20.95,20.78,26.40,29.87,26.21,29.17,26.21,27.40,27.11,29.11,26.77,27.43,26.39,26.59,23.74,27.63,26.45,27.70,25.22,27.62,21.03
pre_test,31.21,28.55,26.93,25.40,32.88,39.30,28.16,28.90,30.64,28.20,27.51,30.09,32.18,33.46,31.90,29.81,31.25,30.14,24.24,29.02,27.37,22.48,23.45,27.51,24.15,21.73,25.95,22.49,21.81,22.19,21.34,24.13,21.84,23.80,25.76,23.10,25.43,24.34,24.29,23.08,21.76,27.94,28.92,27.61,30.34,26.78,29.68,27.34,28.54,30.03,31.43,28.15,28.64,28.32,27.54,28.37,28.78,25.73,28.20,24.33
rec_test,33.62,29.79,26.06,24.76,27.67,37.22,28.43,30.06,29.83,26.30,25.83,26.15,34.62,32.75,32.01,30.37,30.40,29.42,24.80,25.66,25.99,22.83,25.25,27.39,23.34,18.16,23.86,21.12,21.39,21.44,24.15,22.29,19.81,23.04,25.06,21.87,24.71,21.85,23.27,20.95,20.78,26.40,29.87,26.21,29.17,26.21,27.40,27.11,29.11,26.77,27.43,26.39,26.59,23.74,27.63,26.45,27.70,25.22,27.62,21.03
auc_test,39.24,36.44,38.61,39.87,39.95,40.35,37.83,37.98,39.02,37.59,39.54,39.41,39.80,39.60,39.68,40.37,41.27,39.95,32.70,34.95,33.71,34.33,33.39,32.88,31.19,29.29,31.89,31.05,31.88,31.87,30.68,29.33,29.73,32.56,34.38,30.89,32.03,31.26,32.20,31.73,30.83,34.46,37.14,37.25,38.19,39.49,39.48,38.77,34.80,37.33,37.12,36.85,38.18,39.67,35.25,35.74,36.53,35.58,38.21,36.21
f1_test,32.58,29.23,25.69,24.12,30.24,38.89,27.68,29.26,30.06,26.70,26.23,27.23,33.33,32.92,31.81,29.79,30.69,29.56,24.13,26.48,26.40,22.40,24.47,27.48,23.45,19.00,24.46,21.30,21.39,21.79,22.77,23.04,20.36,23.31,25.45,22.43,24.64,22.61,23.72,21.43,21.07,27.41,29.16,26.56,29.63,25.98,28.43,27.04,28.89,28.01,29.01,27.47,27.55,24.70,27.41,27.31,28.18,25.24,27.92,21.63
kappa_test,64.86,59.30,56.75,58.60,63.89,71.66,55.97,60.52,62.47,58.59,60.21,59.31,66.30,67.11,66.51,65.73,68.57,63.50,48.41,53.83,54.06,49.29,53.68,55.26,46.00,41.97,51.09,46.12,49.37,46.93,45.43,47.21,44.57,48.64,54.55,47.12,49.02,48.06,48.90,45.82,46.05,54.08,58.46,56.09,61.67,59.28,61.73,58.24,54.80,56.41,58.99,56.58,59.87,54.58,56.66,57.67,59.92,56.02,62.14,47.48


In [ ]:
rf_3_cls_zero_num.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,16.14,13.15,8.37,6.77,16.53,25.90,9.28,11.53,13.72,9.82,9.77,11.03,14.81,16.32,14.86,12.19,14.49,12.94,6.45,10.63,10.18,5.20,8.37,12.44,6.85,2.02,8.08,3.86,4.57,5.71,3.65,7.12,3.39,6.94,9.81,6.60,7.68,6.30,7.87,4.43,4.63,12.80,11.47,10.01,13.58,8.77,12.77,10.12,13.22,12.81,14.20,12.89,12.32,6.98,10.40,11.65,12.31,8.50,12.16,4.76
pre_test,38.11,35.30,32.88,30.09,40.66,49.70,34.29,34.52,36.72,33.63,32.38,36.15,37.75,39.42,37.33,34.33,35.97,35.39,30.90,36.84,34.24,28.06,28.45,34.10,31.58,28.58,32.79,28.49,26.58,28.10,26.85,31.16,27.87,30.11,31.66,29.82,32.41,31.19,31.18,29.60,27.46,36.41,35.15,33.68,36.49,31.49,35.41,32.60,36.08,38.17,39.55,35.57,34.42,34.41,33.55,34.60,34.62,30.85,33.50,30.85
rec_test,16.14,13.15,8.37,6.77,16.53,25.90,9.28,11.53,13.72,9.82,9.77,11.03,14.81,16.32,14.86,12.19,14.49,12.94,6.45,10.63,10.18,5.20,8.37,12.44,6.85,2.02,8.08,3.86,4.57,5.71,3.65,7.12,3.39,6.94,9.81,6.60,7.68,6.30,7.87,4.43,4.63,12.80,11.47,10.01,13.58,8.77,12.77,10.12,13.22,12.81,14.20,12.89,12.32,6.98,10.40,11.65,12.31,8.50,12.16,4.76
auc_test,41.06,38.10,40.31,41.75,41.50,41.96,38.77,38.92,39.88,38.41,40.46,40.24,40.45,40.18,40.29,40.97,41.86,40.54,34.03,36.18,34.93,35.75,34.70,34.05,32.30,30.42,32.98,32.06,33.06,33.00,31.85,30.46,30.82,33.70,35.46,32.08,33.20,32.45,33.37,32.91,31.95,35.58,38.01,38.19,39.03,40.42,40.24,39.69,35.81,38.38,38.14,38.00,39.17,40.81,36.29,36.73,37.49,36.60,39.26,37.25
f1_test,29.09,26.17,21.33,18.53,30.50,40.71,23.04,24.55,27.08,23.01,22.27,24.76,27.98,29.82,27.86,24.51,26.80,25.82,20.22,25.29,24.09,17.94,20.01,25.37,21.25,16.48,22.13,17.52,16.83,18.73,16.69,21.22,17.09,20.43,22.62,20.12,21.81,20.37,21.54,18.45,17.65,27.04,25.04,23.51,26.98,21.14,25.86,22.93,27.13,27.44,28.77,26.46,25.17,21.10,23.72,25.04,25.34,21.09,24.48,18.88
kappa_test,68.35,62.56,59.92,62.09,67.30,75.04,57.75,62.29,64.22,60.22,62.05,61.02,67.53,68.27,67.73,66.88,69.77,64.69,50.94,56.34,56.52,51.99,56.25,57.65,48.20,44.24,53.28,48.14,51.70,49.20,47.56,49.52,46.76,50.93,56.74,49.51,51.34,50.46,51.24,48.14,48.28,56.37,60.14,57.99,63.40,61.09,63.32,60.05,56.82,58.59,61.13,59.00,61.94,56.76,58.73,59.71,61.89,58.04,64.24,49.58


In [ ]:
rf_3_cls_rand_num.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_3_cls_rand_num.csv')
rf_3_cls_zero_num.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_3_cls_zero_num.csv')

#### Бустинг 3 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_3_cls_boosting_num.csv', index_col=0)

In [ ]:
warnings.simplefilter('ignore')
boost_3_cls_rand_num, boost_3_cls_zero_num = get_effects_neutral(tg_train, tg_test, loggs)
warnings.simplefilter('default')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
boost_3_cls_rand_num.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,37.06,32.17,31.47,29.37,30.41,36.01,29.17,29.94,29.79,27.41,27.45,27.05,33.34,31.97,32.49,29.07,30.04,30.49,28.06,26.26,24.80,25.78,25.76,28.54,23.36,19.51,23.44,19.49,20.90,20.52,23.68,21.56,18.54,20.92,24.56,22.34,23.79,22.18,20.60,20.29,23.14,22.93,28.14,24.83,29.58,25.64,27.68,25.47,26.49,28.72,27.24,27.35,28.52,30.56,25.89,24.79,27.43,26.57,29.08,28.50
pre_test,34.48,30.76,29.78,27.21,34.61,38.14,28.96,29.34,30.98,28.57,28.42,30.70,31.71,33.22,32.52,29.29,31.46,31.01,25.83,28.28,25.29,24.27,24.46,28.41,24.28,22.20,24.69,21.52,22.07,22.59,22.07,23.96,21.62,23.61,25.93,23.80,24.21,24.36,23.85,23.20,23.29,26.08,28.23,28.59,31.11,27.56,30.54,27.25,26.74,30.39,30.57,29.46,29.51,29.54,26.92,27.51,28.63,26.35,29.07,27.56
rec_test,37.06,32.17,31.47,29.37,30.41,36.01,29.17,29.94,29.79,27.41,27.45,27.05,33.34,31.97,32.49,29.07,30.04,30.49,28.06,26.26,24.80,25.78,25.76,28.54,23.36,19.51,23.44,19.49,20.90,20.52,23.68,21.56,18.54,20.92,24.56,22.34,23.79,22.18,20.60,20.29,23.14,22.93,28.14,24.83,29.58,25.64,27.68,25.47,26.49,28.72,27.24,27.35,28.52,30.56,25.89,24.79,27.43,26.57,29.08,28.50
auc_test,40.21,37.64,38.93,40.59,41.50,41.25,38.02,38.54,39.51,38.40,39.94,40.02,39.91,39.78,39.78,40.51,41.57,39.55,33.46,34.45,33.04,35.66,34.47,33.20,30.33,29.36,31.31,30.78,32.03,31.58,29.87,29.63,29.76,32.59,34.57,31.32,30.86,30.38,32.96,32.02,31.14,33.42,37.21,37.84,38.37,40.18,39.79,38.56,35.01,37.96,37.17,37.50,37.93,39.69,34.54,35.58,36.06,36.28,38.09,35.87
f1_test,35.98,31.53,30.58,28.18,32.82,37.71,28.41,29.28,30.10,27.64,27.70,28.04,32.22,32.26,32.31,28.62,30.45,30.58,26.99,26.89,25.01,25.06,25.19,28.27,23.52,20.18,23.88,19.86,21.12,21.23,22.59,22.46,19.30,21.61,25.15,22.98,23.71,22.87,21.57,20.93,23.17,24.28,27.59,25.50,30.09,25.63,28.81,25.70,26.59,29.66,28.71,28.56,29.06,30.14,25.97,25.87,27.95,26.39,29.17,28.03
kappa_test,71.74,63.85,64.80,65.80,68.04,69.72,57.38,60.90,62.81,59.91,62.75,60.64,64.65,66.20,67.65,63.81,68.47,65.44,52.17,53.49,50.44,53.76,55.93,56.47,46.34,43.32,49.25,43.75,49.42,47.11,46.29,46.46,43.46,47.07,54.60,48.74,47.22,48.33,46.69,45.33,49.66,49.76,56.27,55.94,62.92,59.52,63.09,57.02,51.39,58.26,58.01,59.05,61.83,62.58,54.21,55.42,59.53,57.92,64.28,55.27


In [ ]:
boost_3_cls_zero_num.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,19.52,15.54,13.75,11.16,19.32,24.70,9.93,11.47,13.72,10.87,11.36,11.86,13.54,15.59,15.37,10.91,14.06,14.01,9.62,11.09,8.94,8.14,8.82,13.57,6.85,3.25,7.64,2.37,4.13,4.74,3.21,6.34,2.08,4.86,9.20,7.03,6.79,6.59,5.22,3.74,6.89,9.25,9.69,8.55,13.96,8.06,13.04,8.44,10.67,14.78,14.04,13.79,14.29,13.79,8.57,9.96,12.09,9.82,13.55,12.16
pre_test,41.23,37.50,35.79,31.66,42.37,48.56,35.06,34.99,37.07,34.00,33.27,36.77,37.25,39.20,37.96,33.86,36.13,36.23,32.41,36.03,32.14,29.76,29.37,35.01,31.73,29.01,31.47,27.57,26.81,28.45,27.58,30.95,27.63,29.97,31.79,30.41,31.14,31.17,30.74,29.77,29.02,34.49,34.44,34.57,37.28,32.17,36.26,32.45,34.34,38.54,38.69,36.77,35.35,35.64,32.85,33.73,34.54,31.51,34.32,33.99
rec_test,19.52,15.54,13.75,11.16,19.32,24.70,9.93,11.47,13.72,10.87,11.36,11.86,13.54,15.59,15.37,10.91,14.06,14.01,9.62,11.09,8.94,8.14,8.82,13.57,6.85,3.25,7.64,2.37,4.13,4.74,3.21,6.34,2.08,4.86,9.20,7.03,6.79,6.59,5.22,3.74,6.89,9.25,9.69,8.55,13.96,8.06,13.04,8.44,10.67,14.78,14.04,13.79,14.29,13.79,8.57,9.96,12.09,9.82,13.55,12.16
auc_test,41.90,39.31,40.68,42.19,43.04,42.82,38.90,39.49,40.40,39.22,40.81,40.85,40.53,40.37,40.40,41.16,42.11,40.12,34.71,35.61,34.22,36.99,35.71,34.38,31.45,30.49,32.35,31.89,33.18,32.65,31.05,30.73,30.84,33.78,35.60,32.42,31.99,31.52,34.12,33.24,32.28,34.46,38.06,38.69,39.23,41.00,40.55,39.41,36.08,39.00,38.18,38.57,38.98,40.78,35.51,36.54,37.09,37.34,39.07,36.82
f1_test,32.41,28.47,26.23,22.35,33.08,39.56,23.70,24.63,27.15,23.92,23.70,25.54,26.85,29.19,28.38,23.39,26.50,26.84,22.99,25.61,22.65,20.54,20.65,26.15,21.33,17.58,21.52,16.19,16.56,18.12,16.54,20.60,16.00,18.78,22.25,20.59,20.88,20.60,19.41,17.97,19.71,23.85,23.43,22.37,27.43,20.66,26.23,21.54,24.90,29.09,28.48,27.47,26.73,26.55,22.20,23.58,25.17,22.23,25.67,25.20
kappa_test,74.96,67.11,68.07,68.74,71.42,73.07,59.06,62.72,64.59,61.55,64.51,62.37,65.81,67.40,68.88,65.06,69.56,66.59,54.56,55.87,52.84,56.28,58.33,58.89,48.57,45.53,51.34,45.93,51.67,49.25,48.43,48.71,45.61,49.46,56.71,50.95,49.45,50.65,49.04,47.74,51.93,51.93,57.92,57.65,64.68,61.10,64.65,58.70,53.55,60.41,60.14,61.27,64.04,64.73,56.11,57.42,61.66,60.02,66.24,57.19


In [ ]:
boost_3_cls_rand_num.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_3_cls_rand_num.csv')
boost_3_cls_zero_num.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_3_cls_zero_num.csv')

### Кросс-валидация Телеграмм

#### Лес 2 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_2_cls_forest.csv', index_col=0)

In [ ]:
rf_2_cls = get_effects_bin(tg_train, tg_test, loggs)

In [ ]:
rf_2_cls.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_2_cls,AFLT_10_min_2_cls,AFLT_15_min_2_cls,AFLT_30_min_2_cls,AFLT_1_hour_2_cls,AFLT_1_day_2_cls,VTBR_5_min_2_cls,VTBR_10_min_2_cls,VTBR_15_min_2_cls,VTBR_30_min_2_cls,VTBR_1_hour_2_cls,VTBR_1_day_2_cls,GAZP_5_min_2_cls,GAZP_10_min_2_cls,GAZP_15_min_2_cls,GAZP_30_min_2_cls,GAZP_1_hour_2_cls,GAZP_1_day_2_cls,GMKN_5_min_2_cls,GMKN_10_min_2_cls,GMKN_15_min_2_cls,GMKN_30_min_2_cls,GMKN_1_hour_2_cls,GMKN_1_day_2_cls,LKOH_5_min_2_cls,LKOH_10_min_2_cls,LKOH_15_min_2_cls,LKOH_30_min_2_cls,LKOH_1_hour_2_cls,LKOH_1_day_2_cls,ROSN_5_min_2_cls,ROSN_10_min_2_cls,ROSN_15_min_2_cls,ROSN_30_min_2_cls,ROSN_1_hour_2_cls,ROSN_1_day_2_cls,NVTK_5_min_2_cls,NVTK_10_min_2_cls,NVTK_15_min_2_cls,NVTK_30_min_2_cls,NVTK_1_hour_2_cls,NVTK_1_day_2_cls,SBER_5_min_2_cls,SBER_10_min_2_cls,SBER_15_min_2_cls,SBER_30_min_2_cls,SBER_1_hour_2_cls,SBER_1_day_2_cls,TCSG_5_min_2_cls,TCSG_10_min_2_cls,TCSG_15_min_2_cls,TCSG_30_min_2_cls,TCSG_1_hour_2_cls,TCSG_1_day_2_cls,YNDX_5_min_2_cls,YNDX_10_min_2_cls,YNDX_15_min_2_cls,YNDX_30_min_2_cls,YNDX_1_hour_2_cls,YNDX_1_day_2_cls
acc_test,13.12,14.23,10.24,20.08,13.09,19.69,10.94,10.10,11.50,18.66,13.78,19.43,12.95,12.67,12.45,17.85,12.96,18.97,12.92,12.64,8.07,11.50,7.29,14.36,8.24,8.67,5.23,12.56,9.48,13.98,8.13,9.19,7.32,12.49,9.58,12.78,10.08,7.35,6.87,11.24,13.22,14.53,13.05,11.21,11.14,15.49,10.28,18.95,12.12,6.53,4.03,8.12,9.14,15.23,11.93,9.97,8.48,14.61,9.69,14.62
pre_test,10.55,15.52,8.18,25.35,13.41,22.15,11.78,8.78,11.76,20.20,12.08,19.95,14.33,11.85,11.90,21.04,16.03,22.28,11.03,14.21,11.27,15.73,7.86,17.31,8.58,11.26,9.27,14.66,11.20,21.76,8.52,12.34,10.06,15.30,13.33,22.43,10.30,10.29,8.79,12.91,19.29,21.29,16.90,10.56,8.20,17.22,13.12,21.70,12.50,6.58,3.96,16.74,10.40,14.34,14.67,10.88,13.50,15.58,13.65,16.05
rec_test,16.72,8.56,14.22,9.12,9.70,16.51,6.50,13.15,8.85,12.92,17.95,18.20,5.94,13.03,11.75,9.33,2.23,10.15,17.25,6.35,-3.28,-4.77,3.25,2.96,4.39,-2.04,-11.53,4.60,0.52,-8.20,3.90,-3.19,-3.79,1.96,-5.17,-12.02,6.41,-3.76,-0.86,3.97,-1.59,-4.59,2.61,10.71,19.30,9.38,-1.98,12.19,8.19,3.74,2.48,-18.33,-0.04,23.13,0.88,3.49,-7.78,9.12,-7.79,8.63
auc_test,17.02,16.75,13.15,26.33,20.31,26.43,14.70,12.75,15.65,21.36,19.84,22.47,18.07,15.97,17.36,22.46,18.30,23.05,15.72,18.29,11.44,14.88,10.94,18.22,11.53,10.98,9.33,13.61,13.30,16.05,10.74,12.86,10.94,15.72,14.77,17.07,12.11,12.12,11.39,15.64,18.32,18.64,18.10,15.28,14.93,20.19,14.26,22.58,15.59,9.54,6.66,13.66,14.60,19.61,14.79,12.39,13.96,17.55,13.83,18.23
f1_test,13.92,11.88,11.43,16.34,11.63,19.67,9.08,11.03,10.36,16.61,15.02,19.17,10.03,12.50,11.90,14.82,8.59,16.20,14.11,10.29,3.06,4.55,5.57,10.18,6.56,4.03,-3.08,9.52,5.74,4.41,6.31,3.82,2.37,8.33,3.08,1.24,8.37,2.42,3.50,8.51,7.21,6.72,8.85,10.75,13.51,13.36,5.03,16.94,10.58,5.36,3.40,-5.80,5.27,18.63,7.32,7.19,1.14,12.63,1.85,12.50
kappa_test,25.29,29.20,19.66,41.01,26.39,38.92,22.27,19.87,23.17,37.32,27.34,38.86,26.24,25.33,24.93,35.81,26.05,37.03,25.66,25.22,16.77,21.19,14.93,27.46,16.81,18.06,11.76,24.98,18.90,25.99,16.47,19.13,15.77,24.69,19.21,23.27,20.74,16.08,14.82,21.90,27.15,27.44,27.03,22.39,21.25,30.70,20.39,37.51,23.99,12.92,8.00,15.77,17.88,31.06,25.03,20.43,19.12,28.67,19.16,28.84


In [ ]:
rf_2_cls.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_2_cls.csv')

#### Бустинг 2 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_2_cls_boosting.csv', index_col=0)

In [ ]:
boost_2_cls = get_effects_bin(tg_train, tg_test, loggs)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
boost_2_cls.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_2_cls,AFLT_10_min_2_cls,AFLT_15_min_2_cls,AFLT_30_min_2_cls,AFLT_1_hour_2_cls,AFLT_1_day_2_cls,VTBR_5_min_2_cls,VTBR_10_min_2_cls,VTBR_15_min_2_cls,VTBR_30_min_2_cls,VTBR_1_hour_2_cls,VTBR_1_day_2_cls,GAZP_5_min_2_cls,GAZP_10_min_2_cls,GAZP_15_min_2_cls,GAZP_30_min_2_cls,GAZP_1_hour_2_cls,GAZP_1_day_2_cls,GMKN_5_min_2_cls,GMKN_10_min_2_cls,GMKN_15_min_2_cls,GMKN_30_min_2_cls,GMKN_1_hour_2_cls,GMKN_1_day_2_cls,LKOH_5_min_2_cls,LKOH_10_min_2_cls,LKOH_15_min_2_cls,LKOH_30_min_2_cls,LKOH_1_hour_2_cls,LKOH_1_day_2_cls,ROSN_5_min_2_cls,ROSN_10_min_2_cls,ROSN_15_min_2_cls,ROSN_30_min_2_cls,ROSN_1_hour_2_cls,ROSN_1_day_2_cls,NVTK_5_min_2_cls,NVTK_10_min_2_cls,NVTK_15_min_2_cls,NVTK_30_min_2_cls,NVTK_1_hour_2_cls,NVTK_1_day_2_cls,SBER_5_min_2_cls,SBER_10_min_2_cls,SBER_15_min_2_cls,SBER_30_min_2_cls,SBER_1_hour_2_cls,SBER_1_day_2_cls,TCSG_5_min_2_cls,TCSG_10_min_2_cls,TCSG_15_min_2_cls,TCSG_30_min_2_cls,TCSG_1_hour_2_cls,TCSG_1_day_2_cls,YNDX_5_min_2_cls,YNDX_10_min_2_cls,YNDX_15_min_2_cls,YNDX_30_min_2_cls,YNDX_1_hour_2_cls,YNDX_1_day_2_cls
acc_test,11.40,9.11,6.92,20.76,15.01,19.00,9.74,9.65,10.32,18.01,13.52,16.62,12.97,12.32,10.84,15.33,12.15,17.88,10.17,8.26,4.59,11.42,4.27,14.26,6.98,7.25,6.37,11.50,7.16,11.50,5.50,7.29,6.45,11.55,7.36,12.39,10.90,4.89,4.08,9.99,10.76,13.21,12.51,9.87,10.75,14.66,9.51,16.53,7.68,5.52,4.94,9.24,14.44,9.98,9.97,9.20,5.62,13.14,9.33,12.86
pre_test,9.60,9.41,6.27,22.94,13.74,23.62,10.19,9.64,9.63,18.73,12.47,17.51,12.02,11.52,10.45,16.60,12.83,18.37,9.09,9.04,4.35,13.36,3.85,14.82,7.75,7.95,8.53,13.45,8.14,14.18,5.22,8.18,7.72,13.61,9.00,13.29,11.14,5.19,4.31,10.19,15.01,17.69,12.51,9.82,10.04,15.27,9.94,19.70,7.21,5.45,4.78,11.24,16.18,9.75,9.86,10.16,7.42,13.73,11.12,13.88
rec_test,13.55,5.74,7.21,14.16,14.40,10.20,6.37,7.73,10.85,14.16,15.51,14.40,13.30,12.66,9.78,10.48,7.95,16.59,12.42,3.29,5.25,2.76,4.94,11.24,1.39,2.30,-3.95,3.34,-0.08,-1.67,4.81,1.15,-0.50,2.68,-2.51,8.56,7.02,1.65,1.73,9.21,-2.02,-2.28,10.70,7.73,10.55,11.75,5.51,7.87,8.80,3.65,4.02,-0.80,6.19,15.60,6.36,2.59,-4.08,8.46,-1.88,7.54
auc_test,14.41,12.63,12.36,27.21,21.10,25.89,15.84,14.92,13.75,23.49,18.91,20.61,17.87,17.40,16.57,23.04,19.73,23.67,15.62,13.34,9.19,15.07,8.76,19.15,10.82,11.22,9.53,14.90,12.00,15.10,9.89,12.06,11.09,16.89,13.13,18.03,14.26,9.78,8.66,16.47,17.28,17.09,18.40,14.17,15.02,20.86,13.90,20.55,11.76,7.85,7.66,13.06,16.71,12.31,13.29,12.57,11.54,16.07,14.78,17.52
f1_test,11.88,7.65,6.99,18.42,14.36,17.03,8.29,8.79,10.36,16.56,14.08,16.00,12.72,12.15,10.20,13.50,10.43,17.63,10.84,6.27,4.97,8.15,4.56,13.33,4.56,5.14,1.72,8.29,4.05,6.05,5.16,4.66,3.41,8.02,3.02,11.15,9.09,3.50,3.08,9.87,5.37,6.97,11.68,8.84,10.41,13.66,7.84,13.62,8.07,4.76,4.57,5.00,11.29,12.69,8.24,6.36,1.28,11.38,4.54,10.90
kappa_test,22.09,18.61,13.60,41.95,29.63,36.82,19.77,19.40,20.52,36.03,26.92,33.27,25.94,24.63,21.74,30.74,24.36,35.51,20.23,16.48,9.07,21.89,8.37,28.12,14.47,14.82,13.50,22.85,14.27,21.76,11.00,14.94,13.60,22.86,14.75,24.43,22.40,10.13,8.42,19.93,22.17,25.07,25.11,19.91,21.44,29.18,18.96,32.55,15.41,10.96,9.83,18.30,28.54,20.43,20.32,18.90,12.60,25.79,18.52,25.36


In [ ]:
boost_2_cls.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_2_cls.csv')

#### Лес 3 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_3_cls_forest.csv', index_col=0)

In [ ]:
warnings.simplefilter('ignore')
rf_3_cls_rand, rf_3_cls_zero = get_effects_neutral(tg_train, tg_test, loggs)
warnings.simplefilter('default')

In [ ]:
rf_3_cls_rand.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,17.71,15.86,10.67,8.39,19.29,15.76,7.87,4.87,5.42,0.55,6.49,5.36,-3.34,-1.17,-7.40,-10.67,6.88,0.78,9.95,10.65,14.56,13.86,12.01,18.16,2.09,-6.93,-0.67,-1.28,15.43,4.10,-5.58,-2.33,-4.27,-7.10,16.75,7.57,6.53,4.57,8.84,2.35,16.63,11.01,6.32,-5.03,-7.12,-13.37,12.13,-2.92,17.40,12.64,11.62,13.02,16.10,7.81,13.76,10.60,11.23,11.70,12.40,5.14
pre_test,21.18,18.06,15.04,15.89,25.74,22.54,21.52,19.23,22.03,20.81,19.24,22.64,20.04,22.21,21.36,21.46,22.78,22.62,16.42,17.58,18.36,15.82,12.06,21.34,17.66,13.51,16.96,15.73,16.10,15.83,14.81,13.83,14.90,17.58,20.01,17.93,16.87,15.04,16.80,15.68,17.13,20.35,21.72,21.19,21.22,18.58,22.19,20.84,19.49,21.83,21.02,19.27,20.26,14.08,17.83,18.91,18.51,19.83,19.50,16.37
rec_test,17.71,15.86,10.67,8.39,19.29,15.76,7.87,4.87,5.42,0.55,6.49,5.36,-3.34,-1.17,-7.40,-10.67,6.88,0.78,9.95,10.65,14.56,13.86,12.01,18.16,2.09,-6.93,-0.67,-1.28,15.43,4.10,-5.58,-2.33,-4.27,-7.10,16.75,7.57,6.53,4.57,8.84,2.35,16.63,11.01,6.32,-5.03,-7.12,-13.37,12.13,-2.92,17.40,12.64,11.62,13.02,16.10,7.81,13.76,10.60,11.23,11.70,12.40,5.14
auc_test,32.59,28.35,29.45,30.55,33.79,32.32,30.88,30.96,33.12,31.86,32.43,32.59,32.10,33.76,35.06,36.59,35.86,35.68,25.29,26.54,28.36,28.48,25.47,29.46,25.18,24.24,26.88,24.63,26.30,27.28,24.45,23.39,25.56,28.31,28.90,27.82,25.64,25.53,26.25,26.61,25.04,27.66,31.87,30.89,32.31,32.20,32.63,32.42,26.61,28.77,28.18,28.66,30.00,26.79,26.23,27.96,28.61,31.06,29.58,28.37
f1_test,17.64,16.18,11.32,9.07,22.30,18.02,7.21,4.77,6.12,1.28,8.28,6.20,-7.31,-2.72,-10.33,-13.91,8.59,0.66,10.19,11.80,15.44,14.08,11.99,19.29,1.61,-8.35,-0.55,-0.79,15.41,5.63,-8.19,-1.06,-3.47,-8.39,17.89,9.17,6.98,5.76,10.47,3.06,15.73,12.53,5.36,-7.03,-9.89,-16.51,14.24,-3.73,17.93,14.38,13.56,15.10,17.70,8.91,14.58,12.48,12.94,13.06,14.00,5.61
kappa_test,41.66,36.90,31.07,33.49,49.10,39.84,32.96,30.77,33.72,29.86,35.28,35.24,25.04,28.89,24.48,23.16,38.43,32.08,30.22,31.88,36.06,34.46,27.96,42.72,25.87,18.90,25.07,23.93,35.83,29.21,20.24,21.56,21.75,21.55,41.60,32.01,28.77,27.39,31.13,26.26,35.01,35.74,32.73,25.66,24.07,20.50,41.38,28.15,37.59,37.70,36.75,37.31,42.22,28.63,35.28,36.00,36.18,38.32,39.65,29.56


In [ ]:
rf_3_cls_zero.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,0.20,-0.80,-6.97,-9.76,8.17,4.38,-11.36,-13.67,-10.70,-15.81,-9.66,-9.77,-23.15,-17.59,-24.55,-28.88,-9.03,-15.69,-8.48,-4.52,-1.24,-3.85,-4.86,3.28,-14.40,-23.18,-16.42,-18.35,-1.49,-11.68,-26.04,-17.62,-20.66,-23.26,1.48,-7.81,-10.53,-11.02,-6.50,-14.27,0.39,-2.66,-12.12,-21.27,-22.73,-30.90,-2.38,-19.97,1.56,-1.23,-1.64,-0.57,1.81,-8.95,-3.52,-4.25,-4.10,-5.05,-3.08,-11.14
pre_test,28.01,24.88,21.10,20.51,33.40,32.93,27.62,24.83,28.09,26.37,24.01,28.72,25.59,28.14,26.77,25.99,27.52,27.87,23.01,25.25,25.30,21.33,17.06,28.02,25.11,20.34,23.82,21.86,20.76,21.72,20.41,20.72,20.94,23.84,25.94,24.58,23.83,21.82,23.73,22.16,22.90,28.82,27.92,27.16,27.43,23.26,27.98,26.04,27.04,30.00,29.07,26.52,26.02,20.18,23.79,25.12,24.41,24.95,24.79,22.85
rec_test,0.20,-0.80,-6.97,-9.76,8.17,4.38,-11.36,-13.67,-10.70,-15.81,-9.66,-9.77,-23.15,-17.59,-24.55,-28.88,-9.03,-15.69,-8.48,-4.52,-1.24,-3.85,-4.86,3.28,-14.40,-23.18,-16.42,-18.35,-1.49,-11.68,-26.04,-17.62,-20.66,-23.26,1.48,-7.81,-10.53,-11.02,-6.50,-14.27,0.39,-2.66,-12.12,-21.27,-22.73,-30.90,-2.38,-19.97,1.56,-1.23,-1.64,-0.57,1.81,-8.95,-3.52,-4.25,-4.10,-5.05,-3.08,-11.14
auc_test,34.35,30.11,31.26,32.37,35.23,33.94,31.77,31.87,33.97,32.81,33.23,33.43,32.74,34.30,35.66,37.19,36.47,36.28,26.54,27.63,29.63,29.82,26.80,30.73,26.30,25.37,27.97,25.81,27.34,28.37,25.71,24.38,26.67,29.39,30.01,28.94,26.79,26.65,27.45,27.71,26.22,28.78,32.73,31.73,33.21,33.10,33.47,33.26,27.63,29.87,29.14,29.69,30.98,27.90,27.23,28.92,29.65,32.09,30.58,29.36
f1_test,14.12,13.14,7.04,3.37,22.49,19.83,2.51,0.06,3.13,-2.29,4.22,3.73,-12.67,-5.84,-14.29,-19.20,4.70,-3.07,6.21,10.45,13.16,9.53,7.52,17.27,-0.56,-10.93,-2.85,-4.41,10.74,2.53,-14.20,-3.01,-6.72,-11.32,15.06,6.77,4.12,3.47,8.35,0.00,12.29,12.13,1.20,-10.14,-12.53,-21.42,11.76,-7.91,16.21,13.86,13.26,13.96,15.30,5.33,10.84,10.18,10.14,8.89,10.55,2.83
kappa_test,45.02,40.33,34.47,36.83,52.27,43.18,34.65,32.50,35.44,31.77,36.89,36.98,26.24,30.00,25.67,24.31,39.65,33.28,32.62,34.10,38.64,37.00,30.55,45.32,28.10,21.15,27.30,26.26,37.88,31.40,22.54,23.59,23.96,23.73,43.87,34.27,31.05,29.66,33.55,28.47,37.36,38.00,34.37,27.36,25.90,22.24,43.11,29.82,39.65,39.94,38.77,39.43,44.26,30.79,37.25,37.99,38.29,40.35,41.69,31.58


In [ ]:
rf_3_cls_rand.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_3_cls_rand.csv')
rf_3_cls_zero.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_3_cls_zero.csv')

#### Бустинг 3 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_3_cls_boosting.csv', index_col=0)

In [ ]:
warnings.simplefilter('ignore')
boost_3_cls_rand, boost_3_cls_zero = get_effects_neutral(tg_train, tg_test, loggs)
warnings.simplefilter('default')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
boost_3_cls_rand.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,28.56,29.95,28.19,24.39,25.29,34.44,25.88,25.75,25.82,25.20,25.15,25.89,31.16,27.99,28.17,26.91,27.60,27.99,22.47,23.35,23.27,22.95,22.81,20.69,20.49,15.28,19.02,17.64,18.77,16.66,21.38,19.09,16.77,16.91,22.88,20.62,21.40,21.41,19.07,20.80,18.72,23.11,28.35,22.65,25.75,24.97,26.86,25.55,23.26,24.49,22.56,23.69,25.52,26.99,20.81,22.51,24.92,23.08,26.22,20.49
pre_test,28.38,28.39,28.01,24.09,30.97,36.98,27.13,27.21,27.90,26.59,27.16,29.00,30.26,31.20,29.68,28.30,30.02,29.51,23.40,25.20,25.11,21.17,20.97,22.98,22.31,19.96,21.50,19.73,19.72,20.31,20.07,23.03,19.89,21.68,24.11,22.11,22.47,22.75,21.13,21.52,20.09,26.46,28.40,26.70,28.84,26.36,29.71,26.99,23.67,25.65,24.82,24.93,26.61,26.06,23.26,24.80,25.95,23.78,26.91,23.45
rec_test,28.56,29.95,28.19,24.39,25.29,34.44,25.88,25.75,25.82,25.20,25.15,25.89,31.16,27.99,28.17,26.91,27.60,27.99,22.47,23.35,23.27,22.95,22.81,20.69,20.49,15.28,19.02,17.64,18.77,16.66,21.38,19.09,16.77,16.91,22.88,20.62,21.40,21.41,19.07,20.80,18.72,23.11,28.35,22.65,25.75,24.97,26.86,25.55,23.26,24.49,22.56,23.69,25.52,26.99,20.81,22.51,24.92,23.08,26.22,20.49
auc_test,40.63,38.81,41.10,41.02,40.38,41.78,38.13,38.31,39.26,38.45,40.37,39.74,40.86,41.03,41.04,41.70,42.61,41.26,32.76,34.40,33.59,33.09,34.30,32.54,31.56,28.58,29.48,28.94,32.01,31.01,30.85,30.39,30.09,32.93,35.35,32.24,31.53,31.97,31.29,31.06,30.00,33.81,38.40,37.24,39.40,40.65,39.87,38.88,32.09,34.69,33.55,35.19,36.87,37.16,33.09,35.61,36.08,35.00,38.50,34.69
f1_test,27.99,29.25,27.59,23.63,27.99,36.23,25.35,25.45,26.34,25.51,25.60,26.86,30.20,28.61,28.30,26.72,28.23,28.31,22.15,24.01,23.86,22.16,22.01,21.67,20.79,16.30,19.79,18.04,18.96,17.70,20.37,20.31,17.59,18.02,23.44,21.21,21.52,21.92,19.87,20.97,19.17,24.48,27.81,23.40,26.57,24.88,28.00,25.76,23.41,25.26,23.87,24.49,26.13,26.66,21.16,23.49,25.42,23.18,26.57,21.05
kappa_test,57.71,59.18,59.89,56.77,59.69,67.46,52.73,54.83,56.06,55.75,59.05,58.04,61.05,60.44,60.65,60.50,64.47,61.57,45.79,48.03,49.18,47.13,47.86,45.97,42.35,38.04,42.45,40.31,44.53,41.51,42.16,43.95,40.19,41.97,50.87,45.10,43.57,45.71,42.30,43.70,42.50,50.28,56.60,52.26,57.18,57.65,61.44,56.63,45.62,49.38,47.78,49.85,55.84,55.24,46.39,50.54,54.13,51.61,59.08,46.12


In [ ]:
boost_3_cls_zero.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,11.16,13.35,10.56,6.37,14.14,22.91,6.70,7.24,9.71,8.73,8.95,10.76,11.34,11.59,10.99,8.73,11.69,11.51,4.07,8.37,7.47,5.20,6.00,5.77,3.95,-0.97,3.34,0.53,1.93,0.88,0.78,3.82,0.35,0.78,7.64,5.30,4.43,5.81,3.74,4.13,2.46,9.55,9.90,6.39,10.12,7.52,12.28,8.55,7.39,10.51,9.44,10.02,11.33,10.26,3.52,7.69,9.60,6.30,10.77,4.10
pre_test,35.37,35.19,34.11,28.77,38.68,47.22,33.22,32.83,33.99,32.07,32.00,35.08,35.78,37.16,35.08,32.85,34.74,34.76,29.98,33.08,31.92,26.63,25.99,29.60,29.73,26.73,28.37,25.78,24.48,26.20,25.52,29.91,25.93,27.94,30.09,28.75,29.50,29.55,28.07,28.02,25.80,34.98,34.61,32.67,34.97,31.11,35.48,32.25,31.20,33.79,33.04,32.16,32.43,32.12,29.21,31.02,31.83,28.88,32.18,29.90
rec_test,11.16,13.35,10.56,6.37,14.14,22.91,6.70,7.24,9.71,8.73,8.95,10.76,11.34,11.59,10.99,8.73,11.69,11.51,4.07,8.37,7.47,5.20,6.00,5.77,3.95,-0.97,3.34,0.53,1.93,0.88,0.78,3.82,0.35,0.78,7.64,5.30,4.43,5.81,3.74,4.13,2.46,9.55,9.90,6.39,10.12,7.52,12.28,8.55,7.39,10.51,9.44,10.02,11.33,10.26,3.52,7.69,9.60,6.30,10.77,4.10
auc_test,42.54,40.53,42.93,42.90,41.86,43.28,39.02,39.23,40.15,39.32,41.24,40.59,41.48,41.60,41.62,42.33,43.19,41.85,34.01,35.69,34.75,34.36,35.64,33.74,32.66,29.65,30.59,30.04,33.18,32.10,31.95,31.38,31.19,34.00,36.52,33.39,32.75,33.11,32.47,32.21,31.11,34.97,39.27,38.08,40.24,41.62,40.68,39.79,33.09,35.71,34.66,36.20,37.89,38.25,34.07,36.58,37.07,35.99,39.53,35.67
f1_test,24.59,26.24,23.34,18.02,28.20,37.90,20.67,20.77,23.37,21.85,21.55,24.39,24.82,25.52,24.32,21.46,24.33,24.58,18.19,22.87,21.52,17.57,17.58,19.59,18.58,13.68,17.54,14.37,14.39,14.60,14.23,18.36,14.32,15.10,20.66,18.84,18.73,19.63,17.75,17.89,15.70,24.15,23.65,20.27,23.89,20.05,25.49,21.63,21.66,24.67,23.73,23.31,23.80,23.06,17.41,21.22,22.62,18.99,23.11,18.20
kappa_test,61.38,62.57,63.35,60.24,62.97,70.57,54.40,56.60,57.84,57.49,60.80,59.79,62.20,61.60,61.80,61.71,65.65,62.77,48.18,50.65,51.53,49.55,50.48,48.43,44.55,40.16,44.71,42.47,46.82,43.70,44.18,45.97,42.38,44.14,53.24,47.38,45.98,48.01,44.72,45.95,44.75,52.65,58.26,53.96,58.88,59.53,63.12,58.43,47.64,51.51,50.11,51.93,57.98,57.34,48.35,52.56,56.19,53.58,61.12,48.09


In [ ]:
boost_3_cls_rand.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_3_cls_rand.csv')
boost_3_cls_zero.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_3_cls_zero.csv')

### Берт Телеграмм

#### Лес 2 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_2_cls_forest_embs.csv', index_col=0)

In [ ]:
rf_2_cls_embs = get_effects_bin(tg_train, tg_test, loggs)

In [ ]:
rf_2_cls_embs.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_2_cls,AFLT_10_min_2_cls,AFLT_15_min_2_cls,AFLT_30_min_2_cls,AFLT_1_hour_2_cls,AFLT_1_day_2_cls,VTBR_5_min_2_cls,VTBR_10_min_2_cls,VTBR_15_min_2_cls,VTBR_30_min_2_cls,VTBR_1_hour_2_cls,VTBR_1_day_2_cls,GAZP_5_min_2_cls,GAZP_10_min_2_cls,GAZP_15_min_2_cls,GAZP_30_min_2_cls,GAZP_1_hour_2_cls,GAZP_1_day_2_cls,GMKN_5_min_2_cls,GMKN_10_min_2_cls,GMKN_15_min_2_cls,GMKN_30_min_2_cls,GMKN_1_hour_2_cls,GMKN_1_day_2_cls,LKOH_5_min_2_cls,LKOH_10_min_2_cls,LKOH_15_min_2_cls,LKOH_30_min_2_cls,LKOH_1_hour_2_cls,LKOH_1_day_2_cls,ROSN_5_min_2_cls,ROSN_10_min_2_cls,ROSN_15_min_2_cls,ROSN_30_min_2_cls,ROSN_1_hour_2_cls,ROSN_1_day_2_cls,NVTK_5_min_2_cls,NVTK_10_min_2_cls,NVTK_15_min_2_cls,NVTK_30_min_2_cls,NVTK_1_hour_2_cls,NVTK_1_day_2_cls,SBER_5_min_2_cls,SBER_10_min_2_cls,SBER_15_min_2_cls,SBER_30_min_2_cls,SBER_1_hour_2_cls,SBER_1_day_2_cls,TCSG_5_min_2_cls,TCSG_10_min_2_cls,TCSG_15_min_2_cls,TCSG_30_min_2_cls,TCSG_1_hour_2_cls,TCSG_1_day_2_cls,YNDX_5_min_2_cls,YNDX_10_min_2_cls,YNDX_15_min_2_cls,YNDX_30_min_2_cls,YNDX_1_hour_2_cls,YNDX_1_day_2_cls
acc_test,12.21,7.86,6.37,20.60,12.05,18.40,9.57,8.04,10.69,18.41,13.97,17.95,11.91,12.65,11.68,17.83,12.69,18.98,11.27,10.06,2.50,11.76,8.13,13.12,7.97,8.08,4.83,11.54,9.09,12.80,8.32,6.44,5.31,12.01,8.45,13.74,6.23,6.91,5.64,11.88,13.12,14.54,10.87,10.64,10.08,14.79,9.35,18.86,11.58,5.95,6.48,5.64,6.97,10.03,7.77,6.39,5.74,11.30,9.77,14.48
pre_test,10.21,7.96,4.97,23.42,13.41,20.14,9.69,7.04,10.60,19.99,11.71,17.58,13.62,11.88,11.69,21.05,15.90,22.66,10.20,11.50,4.74,16.01,10.12,16.51,8.71,11.45,7.98,13.41,10.34,16.98,8.85,7.24,7.61,14.78,12.30,22.93,7.92,11.55,8.10,14.50,17.43,21.90,12.81,10.77,8.78,16.01,13.23,22.70,11.16,5.94,6.34,10.83,6.00,8.08,11.29,7.18,10.71,12.63,16.35,15.22
rec_test,14.54,5.24,10.01,12.95,6.87,16.43,7.31,10.64,9.05,12.58,20.22,18.79,3.75,12.77,9.15,9.25,1.57,9.44,12.96,3.30,-11.31,-4.28,-0.35,-0.57,2.71,-4.88,-10.01,3.47,1.59,-3.28,3.45,0.64,-5.67,1.20,-7.65,-9.15,-1.08,-8.12,-3.96,1.88,1.34,-5.32,3.81,7.84,12.19,9.98,-6.76,9.90,11.29,3.43,5.12,-18.64,12.36,28.58,-6.48,-0.14,-12.00,2.48,-13.23,10.52
auc_test,13.58,10.31,8.54,23.22,14.79,25.68,13.39,11.37,13.90,20.68,19.32,22.29,16.16,14.57,15.39,19.80,14.80,21.59,14.77,12.95,7.33,9.96,9.45,16.92,10.99,10.93,6.84,12.39,13.09,16.19,10.93,10.87,7.67,13.76,11.92,16.57,10.05,10.79,9.82,14.31,16.33,17.74,15.14,14.81,12.41,18.67,12.08,21.17,15.07,8.83,8.73,10.17,10.62,17.66,11.20,9.15,7.12,14.39,14.22,17.57
f1_test,12.68,6.73,7.75,17.92,9.93,18.65,8.56,8.93,9.91,16.30,15.84,18.31,8.48,12.39,10.49,14.77,8.12,15.95,11.69,7.38,-4.64,5.00,4.52,7.75,5.70,2.27,-2.43,8.37,5.97,6.18,6.24,3.93,0.24,7.67,1.06,3.62,3.00,-0.09,1.27,8.00,8.47,6.42,7.96,9.36,10.59,13.11,2.00,16.08,11.37,4.89,5.91,-7.52,8.92,17.00,1.40,3.51,-2.79,7.74,-1.11,13.07
kappa_test,23.67,15.98,11.90,41.74,24.81,36.49,19.29,15.77,21.46,36.82,27.63,35.84,24.23,25.30,23.49,35.78,25.52,36.99,22.45,20.06,5.89,21.76,17.04,24.69,16.42,17.04,10.81,22.92,18.13,24.13,16.87,13.25,11.80,23.71,16.94,25.44,14.02,15.73,12.72,22.97,26.81,27.41,22.40,21.54,19.82,29.37,18.48,37.21,23.12,11.77,12.90,10.81,14.18,21.46,17.18,13.29,13.99,21.72,19.24,28.68


In [ ]:
rf_2_cls_embs.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_2_cls_embs.csv')

#### Бустинг 2 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_2_cls_boost_embs.csv', index_col=0)

In [ ]:
boost_2_cls_embs = get_effects_bin(tg_train, tg_test, loggs)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
boost_2_cls_embs.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_2_cls,AFLT_10_min_2_cls,AFLT_15_min_2_cls,AFLT_30_min_2_cls,AFLT_1_hour_2_cls,AFLT_1_day_2_cls,VTBR_5_min_2_cls,VTBR_10_min_2_cls,VTBR_15_min_2_cls,VTBR_30_min_2_cls,VTBR_1_hour_2_cls,VTBR_1_day_2_cls,GAZP_5_min_2_cls,GAZP_10_min_2_cls,GAZP_15_min_2_cls,GAZP_30_min_2_cls,GAZP_1_hour_2_cls,GAZP_1_day_2_cls,GMKN_5_min_2_cls,GMKN_10_min_2_cls,GMKN_15_min_2_cls,GMKN_30_min_2_cls,GMKN_1_hour_2_cls,GMKN_1_day_2_cls,LKOH_5_min_2_cls,LKOH_10_min_2_cls,LKOH_15_min_2_cls,LKOH_30_min_2_cls,LKOH_1_hour_2_cls,LKOH_1_day_2_cls,ROSN_5_min_2_cls,ROSN_10_min_2_cls,ROSN_15_min_2_cls,ROSN_30_min_2_cls,ROSN_1_hour_2_cls,ROSN_1_day_2_cls,NVTK_5_min_2_cls,NVTK_10_min_2_cls,NVTK_15_min_2_cls,NVTK_30_min_2_cls,NVTK_1_hour_2_cls,NVTK_1_day_2_cls,SBER_5_min_2_cls,SBER_10_min_2_cls,SBER_15_min_2_cls,SBER_30_min_2_cls,SBER_1_hour_2_cls,SBER_1_day_2_cls,TCSG_5_min_2_cls,TCSG_10_min_2_cls,TCSG_15_min_2_cls,TCSG_30_min_2_cls,TCSG_1_hour_2_cls,TCSG_1_day_2_cls,YNDX_5_min_2_cls,YNDX_10_min_2_cls,YNDX_15_min_2_cls,YNDX_30_min_2_cls,YNDX_1_hour_2_cls,YNDX_1_day_2_cls
acc_test,9.35,11.16,5.61,17.42,10.31,17.38,9.34,8.42,9.20,16.26,12.73,15.87,12.79,11.45,10.03,16.63,12.76,17.56,8.82,6.02,4.48,7.46,5.16,9.69,7.68,7.52,5.55,10.97,8.97,11.98,5.49,8.56,8.00,10.65,7.64,13.95,7.63,6.52,4.78,7.47,8.12,11.07,11.83,9.69,9.79,13.83,8.48,18.05,6.57,8.31,3.89,6.75,6.13,9.03,7.14,5.45,2.94,10.01,9.97,12.62
pre_test,7.90,11.69,6.64,18.62,12.17,19.74,9.01,7.48,8.22,16.95,11.20,15.92,11.99,10.77,9.07,17.20,13.24,18.43,7.43,6.55,5.24,8.60,5.29,10.17,7.92,8.91,6.58,12.35,9.52,12.61,5.43,9.30,8.35,10.89,8.80,15.57,6.55,6.90,5.48,8.29,9.63,13.36,11.70,9.14,9.20,14.26,9.48,22.20,5.60,7.57,3.86,6.81,5.16,8.44,6.44,6.02,3.47,10.38,11.07,13.51
rec_test,11.23,7.17,0.22,12.18,4.12,13.29,8.61,10.59,11.28,12.24,16.70,15.74,12.59,11.52,11.65,13.71,9.35,14.99,13.23,1.55,-0.40,-1.74,2.77,5.10,4.23,0.03,-1.06,4.37,4.27,7.20,3.49,3.28,4.44,8.24,0.11,8.03,8.35,2.97,0.59,1.63,0.95,-0.27,10.42,9.20,9.47,11.50,1.59,7.99,13.78,10.99,1.55,4.56,12.83,17.44,6.69,-0.13,-2.05,5.56,1.72,7.72
auc_test,14.92,13.32,7.91,23.42,14.53,21.94,13.74,11.71,12.97,21.30,18.49,21.45,17.75,16.80,14.86,22.24,18.96,23.25,11.30,10.38,8.69,12.59,7.15,13.45,11.35,10.15,8.21,14.99,12.12,14.53,9.38,12.61,10.98,14.75,13.11,19.32,10.66,9.92,6.70,12.09,12.50,14.95,16.91,14.47,12.77,18.78,12.80,21.87,9.62,10.56,7.33,11.01,9.44,14.50,11.69,8.67,7.02,13.89,12.90,17.68
f1_test,9.89,9.45,3.33,15.40,7.77,16.86,8.92,9.14,9.85,14.71,13.97,15.94,12.36,11.22,10.39,15.50,11.35,16.88,10.31,4.18,2.36,3.52,4.17,7.95,6.17,4.28,2.63,8.37,7.04,10.17,4.60,6.32,6.45,9.72,4.42,11.96,7.64,4.99,2.96,5.09,5.10,6.45,11.14,9.28,9.44,13.03,5.54,14.76,9.26,9.26,2.90,5.85,8.62,12.81,6.70,2.99,0.71,8.23,6.52,10.80
kappa_test,18.02,22.79,11.93,35.21,21.62,34.16,18.68,16.57,18.14,32.52,25.24,31.70,25.60,22.89,19.97,33.31,25.57,34.77,17.42,12.00,9.22,13.83,10.50,18.79,15.65,15.54,11.58,21.80,17.90,23.47,11.07,17.42,16.31,21.23,15.30,27.36,14.85,13.41,10.13,14.44,16.61,21.10,23.72,19.35,19.55,27.55,16.87,35.52,13.58,16.74,7.66,13.46,12.57,18.76,14.29,11.35,6.59,19.54,19.83,24.89


In [ ]:
boost_2_cls_embs.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_2_cls_embs.csv')

#### Лес 3 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_3_cls_forest_embs.csv', index_col=0)

In [ ]:
warnings.simplefilter('ignore')
rf_3_cls_emb_rand, rf_3_cls_emb_zero = get_effects_neutral(tg_train, tg_test, loggs)
warnings.simplefilter('default')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
rf_3_cls_emb_rand.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,19.35,13.43,16.78,15.64,12.32,11.39,20.74,17.60,17.21,15.93,15.64,17.37,10.87,12.98,13.16,9.93,16.86,15.20,20.30,16.78,16.34,18.17,17.16,20.56,16.06,14.89,16.67,16.56,15.88,16.82,17.86,14.47,11.74,14.75,14.80,14.84,15.71,16.69,15.50,14.49,16.71,15.82,14.15,15.59,16.32,11.56,16.84,18.16,15.41,15.08,14.02,13.37,16.05,18.94,16.66,13.97,15.69,15.71,15.71,13.18
pre_test,13.25,6.42,9.15,4.39,17.77,7.77,16.71,13.79,17.04,12.11,17.05,17.88,16.87,18.45,18.77,18.57,19.00,20.39,14.13,14.96,13.45,11.99,13.31,19.37,11.94,9.48,12.67,10.56,10.95,13.02,11.19,11.47,9.87,11.92,11.80,11.99,11.53,13.14,10.95,7.74,14.07,15.63,15.41,15.18,15.71,9.93,15.88,17.09,8.61,11.16,12.70,7.00,15.74,16.02,8.43,11.10,11.39,13.51,13.71,16.61
rec_test,19.35,13.43,16.78,15.64,12.32,11.39,20.74,17.60,17.21,15.93,15.64,17.37,10.87,12.98,13.16,9.93,16.86,15.20,20.30,16.78,16.34,18.17,17.16,20.56,16.06,14.89,16.67,16.56,15.88,16.82,17.86,14.47,11.74,14.75,14.80,14.84,15.71,16.69,15.50,14.49,16.71,15.82,14.15,15.59,16.32,11.56,16.84,18.16,15.41,15.08,14.02,13.37,16.05,18.94,16.66,13.97,15.69,15.71,15.71,13.18
auc_test,22.51,18.39,19.03,19.08,24.11,23.82,26.88,23.00,26.41,23.21,28.20,28.49,27.22,27.74,28.77,31.14,29.48,31.19,22.12,19.76,18.38,19.04,22.36,24.18,18.39,16.81,19.35,17.68,18.93,18.51,20.73,17.76,18.41,19.89,20.10,20.65,18.72,20.38,21.42,16.87,20.01,21.76,24.86,25.30,25.40,23.06,27.25,27.59,18.05,18.93,16.76,16.80,21.07,24.29,19.08,20.70,19.85,22.18,23.86,25.55
f1_test,11.25,8.32,11.28,7.51,4.16,2.39,18.90,15.74,17.17,12.91,16.17,17.66,10.77,14.14,14.11,10.90,17.63,16.32,15.01,10.44,5.89,12.32,13.79,18.20,12.37,10.06,12.21,11.66,11.84,10.69,14.61,11.72,10.81,13.02,12.44,12.78,12.66,11.99,9.39,8.73,13.60,9.30,14.13,15.40,16.00,10.82,15.74,17.57,3.20,9.02,5.15,3.50,9.13,14.13,7.76,11.04,9.79,14.54,14.20,13.97
kappa_test,12.96,8.84,14.52,4.86,4.91,2.15,34.13,28.45,35.09,21.45,38.07,36.41,32.13,35.91,37.18,36.03,41.69,41.09,21.49,14.07,3.97,17.01,26.46,33.24,18.08,13.26,18.55,16.29,20.23,14.36,22.98,18.37,20.24,23.08,21.63,22.00,19.57,17.89,13.16,9.97,24.92,13.31,31.41,31.16,31.59,22.43,30.44,35.88,-0.10,12.61,6.01,2.10,12.40,23.14,6.42,17.78,13.14,28.79,27.96,33.24


In [ ]:
rf_3_cls_emb_zero.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,1.59,-3.19,-0.80,-2.39,1.20,0.00,1.48,-0.99,1.10,-0.55,-0.49,2.20,-8.93,-3.45,-4.00,-8.28,0.90,-1.30,1.92,1.70,0.57,0.57,0.34,5.66,-0.44,-1.32,0.88,-0.61,-0.97,0.97,-2.69,-0.78,-4.69,-1.30,-0.52,-0.52,-1.28,1.18,0.20,-2.07,0.39,2.17,-4.33,-0.65,0.70,-5.95,2.27,1.14,-0.41,1.15,0.82,-0.25,1.72,2.22,-0.66,-0.81,0.29,-0.95,0.29,-3.15
pre_test,19.93,13.16,15.26,9.05,25.52,18.14,22.81,19.35,23.08,17.58,21.87,23.90,22.41,24.39,24.17,23.09,23.70,25.63,20.76,22.78,20.40,17.50,18.26,26.05,19.35,16.34,19.45,16.54,15.70,18.87,16.75,18.40,15.87,18.24,17.69,18.57,18.56,20.02,17.89,14.27,19.73,24.10,21.62,21.17,21.87,14.64,21.62,22.34,16.19,19.26,20.79,14.22,21.56,22.14,14.40,17.29,17.23,18.66,19.03,23.12
rec_test,1.59,-3.19,-0.80,-2.39,1.20,0.00,1.48,-0.99,1.10,-0.55,-0.49,2.20,-8.93,-3.45,-4.00,-8.28,0.90,-1.30,1.92,1.70,0.57,0.57,0.34,5.66,-0.44,-1.32,0.88,-0.61,-0.97,0.97,-2.69,-0.78,-4.69,-1.30,-0.52,-0.52,-1.28,1.18,0.20,-2.07,0.39,2.17,-4.33,-0.65,0.70,-5.95,2.27,1.14,-0.41,1.15,0.82,-0.25,1.72,2.22,-0.66,-0.81,0.29,-0.95,0.29,-3.15
auc_test,24.12,20.06,20.85,20.96,25.64,25.40,27.78,23.85,27.23,24.06,29.05,29.27,27.83,28.30,29.35,31.74,30.05,31.77,23.42,20.99,19.66,20.38,23.63,25.44,19.48,17.97,20.36,18.74,20.09,19.55,21.93,18.80,19.48,21.03,21.16,21.72,19.93,21.61,22.63,18.06,21.09,22.87,25.72,26.15,26.25,23.99,28.01,28.49,19.10,19.92,17.77,17.78,22.09,25.44,20.09,21.65,20.82,23.25,24.90,26.58
f1_test,7.52,5.28,7.06,1.89,4.41,4.19,14.18,10.98,14.17,9.24,12.13,15.14,5.41,11.03,10.15,5.61,13.70,12.58,11.08,9.22,3.64,7.83,9.33,16.17,10.16,7.50,9.85,7.92,7.26,7.53,8.53,9.82,7.52,10.18,9.57,10.35,9.86,9.79,7.29,5.74,10.07,8.90,9.96,12.30,13.34,5.94,13.21,13.43,1.50,8.43,4.91,2.34,6.75,10.57,4.00,8.77,6.94,10.44,10.78,11.17
kappa_test,16.02,12.11,17.99,8.31,8.27,5.47,35.82,30.08,36.75,23.17,39.77,38.05,33.30,37.04,38.33,37.18,42.85,42.27,23.98,16.58,6.57,19.56,28.94,35.82,20.25,15.57,20.63,18.32,22.50,16.46,25.18,20.51,22.38,25.38,23.81,24.14,21.97,20.36,15.59,12.30,27.05,15.57,33.06,32.88,33.34,24.22,32.03,37.65,2.03,14.67,8.12,4.17,14.54,25.37,8.40,19.75,15.12,30.89,30.06,35.29


In [ ]:
rf_3_cls_emb_rand.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_3_cls_emb_rand.csv')
rf_3_cls_emb_zero.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_3_cls_emb_zero.csv')

#### Бустинг 3 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_3_cls_boost_embs.csv', index_col=0)

In [ ]:
warnings.simplefilter('ignore')
boost_3_cls_emb_rand, boost_3_cls_emb_zero = get_effects_neutral(tg_train, tg_test, loggs)
warnings.simplefilter('default')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
boost_3_cls_emb_rand.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,34.78,32.45,28.44,26.57,28.49,31.83,31.48,30.69,28.64,26.77,27.95,30.24,33.11,31.23,31.51,29.49,30.09,30.81,27.58,26.19,24.80,24.86,24.50,25.34,23.13,22.95,23.74,21.91,23.26,21.46,25.42,23.60,21.93,23.02,25.96,23.05,25.21,22.94,24.16,23.90,23.31,25.22,29.84,25.15,28.39,27.47,27.70,29.82,24.03,27.24,26.03,23.84,24.26,27.67,26.49,25.73,26.83,26.11,28.48,25.35
pre_test,31.87,30.38,25.69,22.42,32.30,33.84,29.04,28.52,28.44,26.39,27.94,30.88,30.58,32.35,31.10,28.84,31.00,30.90,23.57,25.55,23.70,21.01,21.53,25.39,20.97,21.46,22.45,19.76,21.02,19.90,20.17,23.70,20.98,22.51,25.41,21.98,22.60,21.81,23.10,21.89,21.62,25.48,27.89,25.56,29.13,26.78,29.03,28.40,23.28,27.69,27.34,24.54,25.20,26.22,24.12,25.69,26.36,24.53,27.88,23.93
rec_test,34.78,32.45,28.44,26.57,28.49,31.83,31.48,30.69,28.64,26.77,27.95,30.24,33.11,31.23,31.51,29.49,30.09,30.81,27.58,26.19,24.80,24.86,24.50,25.34,23.13,22.95,23.74,21.91,23.26,21.46,25.42,23.60,21.93,23.02,25.96,23.05,25.21,22.94,24.16,23.90,23.31,25.22,29.84,25.15,28.39,27.47,27.70,29.82,24.03,27.24,26.03,23.84,24.26,27.67,26.49,25.73,26.83,26.11,28.48,25.35
auc_test,39.02,36.29,36.89,40.10,38.63,39.60,37.74,37.37,37.90,36.53,38.79,39.24,39.15,39.05,39.14,40.07,40.73,39.36,32.50,33.22,31.30,30.45,30.29,30.32,28.22,27.04,30.73,28.28,29.47,29.42,28.43,28.59,28.94,31.22,32.38,29.71,28.61,28.86,29.65,29.37,28.24,31.46,36.54,36.11,37.36,38.83,38.39,37.88,29.75,34.46,32.58,32.73,35.95,37.05,32.42,33.66,34.33,33.95,37.15,34.60
f1_test,33.27,31.35,27.09,24.51,30.13,33.19,30.32,29.68,28.59,26.60,27.94,30.61,31.84,31.50,31.25,28.90,30.41,30.79,25.41,25.33,23.78,22.73,22.71,23.86,21.81,22.13,22.91,20.90,22.17,19.86,23.17,23.65,21.53,22.82,25.51,22.39,23.44,22.29,23.32,22.83,22.25,25.07,28.91,25.28,28.73,26.98,28.45,29.18,21.22,26.74,25.79,23.09,23.02,25.46,25.27,25.36,26.26,25.32,28.09,24.51
kappa_test,64.95,62.34,56.00,54.67,61.27,60.72,59.37,60.48,58.45,56.29,62.38,63.07,63.22,64.65,65.07,63.71,67.99,65.47,46.47,46.60,44.88,45.11,47.40,45.33,39.22,42.79,44.03,41.50,47.88,38.19,43.67,46.23,43.88,46.29,52.74,44.00,42.40,43.32,45.20,44.02,45.36,46.86,57.02,52.57,59.50,60.43,60.86,61.49,36.27,49.93,48.55,44.53,46.13,49.16,49.62,50.97,53.53,53.68,60.65,47.39


In [ ]:
boost_3_cls_emb_zero.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,17.13,15.94,10.76,8.57,17.33,20.52,12.24,12.13,12.62,10.37,11.80,15.04,13.31,14.81,14.36,11.31,14.19,14.34,9.16,11.09,9.05,7.13,7.69,10.41,6.58,6.76,7.99,4.83,6.32,5.62,4.95,8.42,5.47,6.94,10.68,7.64,8.17,7.38,8.86,7.28,7.19,11.61,11.42,8.93,12.82,9.96,13.10,12.77,8.29,13.22,12.81,10.18,9.93,10.92,9.23,10.92,11.43,9.38,13.11,9.01
pre_test,38.56,37.27,31.70,26.99,40.00,44.29,35.08,34.09,34.59,31.95,32.76,36.91,36.12,38.30,36.49,33.40,35.71,36.15,30.14,33.32,30.59,26.50,26.55,32.05,28.36,28.28,29.28,25.84,25.69,25.72,25.73,30.70,27.02,28.84,31.28,28.54,29.59,28.66,29.96,28.35,27.40,33.98,34.10,31.58,35.33,31.44,34.81,33.56,30.89,35.78,35.43,31.79,30.95,32.27,30.10,31.91,32.15,29.67,33.24,30.39
rec_test,17.13,15.94,10.76,8.57,17.33,20.52,12.24,12.13,12.62,10.37,11.80,15.04,13.31,14.81,14.36,11.31,14.19,14.34,9.16,11.09,9.05,7.13,7.69,10.41,6.58,6.76,7.99,4.83,6.32,5.62,4.95,8.42,5.47,6.94,10.68,7.64,8.17,7.38,8.86,7.28,7.19,11.61,11.42,8.93,12.82,9.96,13.10,12.77,8.29,13.22,12.81,10.18,9.93,10.92,9.23,10.92,11.43,9.38,13.11,9.01
auc_test,40.63,38.10,38.62,41.87,40.10,41.24,38.59,38.23,38.83,37.49,39.65,40.03,39.76,39.62,39.71,40.71,41.32,39.96,33.75,34.42,32.54,31.78,31.65,31.56,29.30,28.15,31.80,29.43,30.55,30.44,29.64,29.69,30.04,32.37,33.42,30.78,29.79,30.02,30.77,30.48,29.43,32.61,37.40,37.00,38.25,39.68,39.19,38.68,30.82,35.46,33.57,33.72,36.92,38.17,33.45,34.63,35.25,34.97,38.23,35.57
f1_test,29.60,28.42,22.78,18.86,30.33,35.06,25.59,24.94,25.68,23.02,23.90,28.08,26.48,28.40,27.29,23.65,26.52,27.06,21.42,24.08,21.50,18.16,18.29,21.80,19.57,19.57,20.59,17.26,17.49,16.68,17.14,21.81,18.24,19.97,22.65,19.93,20.60,20.05,21.18,19.77,18.89,24.71,24.77,22.20,26.11,22.07,25.91,24.98,19.57,26.10,25.53,21.92,20.60,21.85,21.55,23.08,23.39,21.17,24.72,21.69
kappa_test,68.05,65.90,59.27,57.91,64.54,64.15,60.97,62.14,60.34,58.21,64.09,64.72,64.38,65.80,66.22,64.95,69.18,66.67,48.86,49.04,47.38,47.63,50.03,47.87,41.35,45.01,46.20,43.74,49.99,40.25,45.88,48.50,46.07,48.61,54.87,46.11,44.74,45.71,47.48,46.20,47.75,49.19,58.68,54.36,61.33,62.10,62.54,63.08,38.42,51.98,50.63,46.62,48.14,51.27,51.64,52.99,55.42,55.73,62.82,49.36


In [ ]:
boost_3_cls_emb_rand.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_3_cls_emb_rand.csv')
boost_3_cls_emb_zero.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_3_cls_emb_zero.csv')

### Все вместе Телеграмм

#### Лес 2 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_2_cls_forest_all.csv', index_col=0)

In [ ]:
rf_2_cls_all = get_effects_bin(tg_train, tg_test, loggs)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
rf_2_cls_all.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_2_cls,AFLT_10_min_2_cls,AFLT_15_min_2_cls,AFLT_30_min_2_cls,AFLT_1_hour_2_cls,AFLT_1_day_2_cls,VTBR_5_min_2_cls,VTBR_10_min_2_cls,VTBR_15_min_2_cls,VTBR_30_min_2_cls,VTBR_1_hour_2_cls,VTBR_1_day_2_cls,GAZP_5_min_2_cls,GAZP_10_min_2_cls,GAZP_15_min_2_cls,GAZP_30_min_2_cls,GAZP_1_hour_2_cls,GAZP_1_day_2_cls,GMKN_5_min_2_cls,GMKN_10_min_2_cls,GMKN_15_min_2_cls,GMKN_30_min_2_cls,GMKN_1_hour_2_cls,GMKN_1_day_2_cls,LKOH_5_min_2_cls,LKOH_10_min_2_cls,LKOH_15_min_2_cls,LKOH_30_min_2_cls,LKOH_1_hour_2_cls,LKOH_1_day_2_cls,ROSN_5_min_2_cls,ROSN_10_min_2_cls,ROSN_15_min_2_cls,ROSN_30_min_2_cls,ROSN_1_hour_2_cls,ROSN_1_day_2_cls,NVTK_5_min_2_cls,NVTK_10_min_2_cls,NVTK_15_min_2_cls,NVTK_30_min_2_cls,NVTK_1_hour_2_cls,NVTK_1_day_2_cls,SBER_5_min_2_cls,SBER_10_min_2_cls,SBER_15_min_2_cls,SBER_30_min_2_cls,SBER_1_hour_2_cls,SBER_1_day_2_cls,TCSG_5_min_2_cls,TCSG_10_min_2_cls,TCSG_15_min_2_cls,TCSG_30_min_2_cls,TCSG_1_hour_2_cls,TCSG_1_day_2_cls,YNDX_5_min_2_cls,YNDX_10_min_2_cls,YNDX_15_min_2_cls,YNDX_30_min_2_cls,YNDX_1_hour_2_cls,YNDX_1_day_2_cls
acc_test,12.33,6.32,5.45,17.26,8.39,15.32,10.14,9.00,10.95,18.47,13.65,16.83,10.59,12.58,9.63,18.07,12.98,18.57,13.04,8.79,3.19,11.76,5.92,12.32,8.46,8.67,4.53,11.31,8.33,10.42,8.68,7.04,6.54,11.75,7.82,12.68,6.42,7.91,6.27,11.30,12.69,13.84,12.80,10.95,10.70,14.88,7.98,18.76,8.27,4.44,1.68,6.84,7.26,9.74,9.21,6.65,4.91,12.64,9.41,13.36
pre_test,10.18,7.41,3.80,20.08,10.50,16.16,10.70,7.83,9.88,20.10,11.73,16.88,15.17,11.80,14.32,20.93,16.13,21.79,11.77,9.19,5.61,15.48,6.93,15.04,8.70,11.59,7.92,13.34,10.83,14.07,9.08,8.29,9.21,14.41,11.94,19.34,7.37,11.88,7.74,13.78,17.64,19.53,14.52,10.78,8.28,16.77,11.69,22.14,8.39,4.53,1.84,10.84,7.69,7.84,10.98,7.33,9.69,13.20,14.30,14.02
rec_test,15.20,1.07,10.81,9.24,1.59,15.75,6.44,11.94,12.72,12.56,18.79,16.79,-4.99,12.78,-6.54,10.31,2.08,9.76,14.88,5.78,-10.42,-2.72,-0.00,-0.36,4.89,-2.86,-11.22,2.63,-4.13,-7.29,4.44,-0.56,-4.79,1.13,-9.51,-7.42,1.16,-5.23,-0.29,1.26,-0.47,-3.81,6.69,9.06,17.07,8.14,-9.16,10.67,4.93,0.92,-6.72,-12.21,1.40,28.13,-0.06,0.64,-13.24,7.97,-10.55,9.52
auc_test,14.23,11.03,8.73,20.61,13.04,20.79,13.55,11.77,14.49,19.31,18.72,19.67,15.67,15.02,15.73,20.70,14.02,22.84,15.29,14.20,9.49,13.17,8.24,14.99,10.44,10.61,6.24,12.01,12.18,11.97,10.60,12.19,9.50,13.79,10.08,15.72,10.44,10.26,9.67,13.43,15.47,17.15,16.20,13.66,12.62,17.70,11.56,21.83,13.40,6.36,4.95,9.91,10.70,15.60,11.96,9.76,7.29,15.81,13.21,16.82
f1_test,13.01,4.09,7.48,14.33,5.56,16.22,8.57,9.97,11.40,16.36,15.22,16.95,3.68,12.35,2.24,15.33,8.53,15.76,13.42,7.65,-3.74,5.77,3.35,7.26,6.89,3.64,-3.29,7.84,2.83,2.53,6.86,3.74,1.41,7.48,-0.34,3.85,4.10,2.03,3.40,7.34,7.35,6.65,10.35,10.01,12.57,12.47,-0.16,16.29,6.89,2.94,-2.31,-2.56,4.77,16.67,5.22,4.02,-4.01,10.86,0.20,11.98
kappa_test,23.81,13.59,9.77,35.16,18.00,30.68,20.59,17.65,21.68,36.95,27.04,33.61,21.97,25.16,20.12,36.24,26.10,36.23,25.99,17.56,7.25,21.92,12.40,23.17,17.23,18.13,10.29,22.45,16.57,19.14,17.57,14.54,14.26,23.20,15.69,23.51,13.91,17.44,13.45,21.81,26.02,26.17,26.12,22.03,20.57,29.45,15.71,37.06,16.34,8.72,2.88,13.33,14.25,20.87,19.45,13.77,12.42,24.78,18.56,26.45


In [ ]:
rf_2_cls_all.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_2_cls_all.csv')

#### Бустинг 2 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_2_cls_boost_all.csv', index_col=0)

In [ ]:
boost_2_cls_all = get_effects_bin(tg_train, tg_test, loggs)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
boost_2_cls_all.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_2_cls,AFLT_10_min_2_cls,AFLT_15_min_2_cls,AFLT_30_min_2_cls,AFLT_1_hour_2_cls,AFLT_1_day_2_cls,VTBR_5_min_2_cls,VTBR_10_min_2_cls,VTBR_15_min_2_cls,VTBR_30_min_2_cls,VTBR_1_hour_2_cls,VTBR_1_day_2_cls,GAZP_5_min_2_cls,GAZP_10_min_2_cls,GAZP_15_min_2_cls,GAZP_30_min_2_cls,GAZP_1_hour_2_cls,GAZP_1_day_2_cls,GMKN_5_min_2_cls,GMKN_10_min_2_cls,GMKN_15_min_2_cls,GMKN_30_min_2_cls,GMKN_1_hour_2_cls,GMKN_1_day_2_cls,LKOH_5_min_2_cls,LKOH_10_min_2_cls,LKOH_15_min_2_cls,LKOH_30_min_2_cls,LKOH_1_hour_2_cls,LKOH_1_day_2_cls,ROSN_5_min_2_cls,ROSN_10_min_2_cls,ROSN_15_min_2_cls,ROSN_30_min_2_cls,ROSN_1_hour_2_cls,ROSN_1_day_2_cls,NVTK_5_min_2_cls,NVTK_10_min_2_cls,NVTK_15_min_2_cls,NVTK_30_min_2_cls,NVTK_1_hour_2_cls,NVTK_1_day_2_cls,SBER_5_min_2_cls,SBER_10_min_2_cls,SBER_15_min_2_cls,SBER_30_min_2_cls,SBER_1_hour_2_cls,SBER_1_day_2_cls,TCSG_5_min_2_cls,TCSG_10_min_2_cls,TCSG_15_min_2_cls,TCSG_30_min_2_cls,TCSG_1_hour_2_cls,TCSG_1_day_2_cls,YNDX_5_min_2_cls,YNDX_10_min_2_cls,YNDX_15_min_2_cls,YNDX_30_min_2_cls,YNDX_1_hour_2_cls,YNDX_1_day_2_cls
acc_test,9.87,9.50,9.09,17.68,12.27,18.48,10.21,8.92,10.66,16.41,13.61,16.51,12.68,11.78,11.61,16.29,13.32,16.90,8.39,7.57,2.84,10.01,4.82,11.82,6.91,5.47,5.36,10.02,8.49,10.28,5.33,9.54,8.41,11.17,10.88,12.93,8.51,6.98,4.48,7.72,12.60,13.75,13.29,11.25,10.57,13.83,7.85,14.97,7.88,8.11,3.73,9.30,10.28,8.68,9.19,5.98,4.26,11.36,9.91,12.93
pre_test,9.32,8.03,8.33,19.10,12.11,24.28,10.94,8.44,10.50,16.83,12.43,16.28,11.90,10.82,10.45,17.61,13.63,17.12,7.65,8.56,3.78,11.48,4.52,12.82,7.80,6.66,6.83,11.16,9.16,11.97,5.17,10.99,8.44,12.03,12.37,13.95,8.10,7.38,4.44,8.41,16.36,18.46,13.06,10.64,9.56,14.14,8.43,17.91,6.92,8.01,3.61,9.32,9.56,8.45,8.06,7.08,4.90,11.70,12.29,13.42
rec_test,9.09,10.73,9.13,11.84,9.98,6.60,6.11,8.92,9.34,13.22,16.04,17.19,12.47,12.91,13.49,11.45,10.55,16.36,9.72,1.27,-4.11,1.97,4.60,5.74,0.90,-2.17,-3.23,3.79,3.13,-0.33,4.09,2.06,6.04,6.34,3.66,8.83,6.79,3.44,3.21,2.89,1.61,-1.76,12.09,10.64,11.45,11.92,2.71,5.98,13.27,6.44,2.78,7.49,11.42,14.58,9.61,-1.94,-1.32,7.40,-2.85,9.69
auc_test,15.35,13.06,13.92,25.66,16.42,23.33,15.48,13.12,14.14,22.65,19.44,20.67,17.89,16.83,16.78,23.55,19.54,23.16,13.36,13.20,7.44,14.01,7.53,16.27,11.00,7.74,7.80,15.16,12.60,13.82,8.91,14.17,11.68,16.22,15.01,18.68,11.55,10.36,8.12,14.08,15.60,15.88,18.16,15.38,14.04,19.95,12.82,19.86,12.01,9.75,6.07,12.69,13.75,14.03,12.44,10.19,9.10,16.35,14.30,17.09
f1_test,9.43,9.68,8.99,15.44,11.19,15.22,8.48,8.79,10.01,15.14,14.32,16.84,12.26,11.90,11.99,14.50,12.16,16.86,8.81,4.94,-0.38,6.86,4.74,9.59,4.30,2.07,1.50,7.53,6.26,5.88,4.78,6.41,7.35,9.31,8.00,11.60,7.57,5.46,3.95,5.82,8.23,7.58,12.67,10.75,10.62,13.17,5.66,11.76,9.84,7.41,3.36,8.56,10.54,11.55,8.94,2.46,1.78,9.82,4.44,11.75
kappa_test,19.59,18.36,17.95,35.79,24.58,35.33,20.78,17.78,21.37,32.83,27.09,32.96,25.37,23.52,23.13,32.65,26.69,33.62,16.71,15.07,6.10,19.11,9.57,22.90,14.38,11.47,11.37,19.91,16.93,19.53,10.71,19.52,17.00,22.21,21.79,25.48,17.17,14.33,9.04,15.05,25.73,26.16,26.61,22.49,20.95,27.57,15.62,29.38,16.07,16.14,7.43,18.58,20.59,17.88,18.32,12.60,9.29,22.29,19.65,25.63


In [ ]:
boost_2_cls_all.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_2_cls_all.csv')

#### Лес 3 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_3_cls_forest_all.csv', index_col=0)

In [ ]:
warnings.simplefilter('ignore')
rf_3_cls_all_rand, rf_3_cls_all_zero = get_effects_neutral(tg_train, tg_test, loggs)
warnings.simplefilter('default')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
rf_3_cls_all_rand.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,18.84,16.77,15.06,17.11,14.41,10.32,17.86,12.92,15.59,14.71,15.82,13.71,5.14,9.22,7.26,3.42,11.61,8.37,20.08,17.78,17.84,18.07,13.06,19.05,15.82,10.60,16.11,14.28,16.32,14.61,17.29,14.75,13.28,13.89,14.05,17.90,18.53,15.58,16.58,15.71,17.53,17.15,14.71,15.42,12.75,9.64,17.98,16.04,16.24,16.57,15.17,14.37,14.16,16.40,15.78,16.58,16.92,17.98,15.71,14.11
pre_test,13.12,10.26,6.59,-0.68,20.02,8.44,16.81,13.97,15.60,12.56,15.19,15.10,16.39,18.05,17.86,18.16,18.23,19.45,13.94,16.39,15.44,11.40,10.94,17.73,12.36,10.63,13.73,10.99,10.30,10.70,13.59,12.41,11.86,11.82,15.09,16.40,14.36,14.01,13.33,13.33,14.71,16.01,18.12,16.05,15.51,13.34,18.91,15.64,12.19,14.34,16.50,11.60,11.30,11.45,6.10,14.08,14.02,15.61,11.03,16.06
rec_test,18.84,16.77,15.06,17.11,14.41,10.32,17.86,12.92,15.59,14.71,15.82,13.71,5.14,9.22,7.26,3.42,11.61,8.37,20.08,17.78,17.84,18.07,13.06,19.05,15.82,10.60,16.11,14.28,16.32,14.61,17.29,14.75,13.28,13.89,14.05,17.90,18.53,15.58,16.58,15.71,17.53,17.15,14.71,15.42,12.75,9.64,17.98,16.04,16.24,16.57,15.17,14.37,14.16,16.40,15.78,16.58,16.92,17.98,15.71,14.11
auc_test,25.43,19.67,16.07,19.24,23.91,22.57,26.60,23.85,25.06,22.89,27.56,25.46,26.97,27.53,28.28,30.02,30.01,30.04,21.98,21.36,20.17,21.12,20.81,21.95,18.51,18.57,21.39,18.50,19.74,19.96,20.60,17.80,19.79,20.81,23.08,24.14,19.79,21.51,20.87,20.33,20.91,23.77,27.54,25.30,25.20,26.31,29.44,26.23,18.04,20.30,18.69,17.44,20.37,22.33,19.97,21.25,22.57,25.22,22.61,25.20
f1_test,14.39,9.28,9.71,6.45,9.93,7.06,17.19,12.93,15.64,13.47,15.55,14.40,4.51,10.37,8.31,4.16,13.20,9.80,14.98,15.67,16.26,12.92,11.98,15.81,13.42,10.60,14.44,12.53,9.92,11.39,15.69,12.60,12.64,12.77,14.54,16.98,14.99,14.71,12.46,14.50,12.14,14.85,14.79,15.62,13.77,10.42,18.53,15.83,5.55,10.73,7.19,6.82,10.63,11.81,6.68,11.53,13.22,16.62,12.03,14.66
kappa_test,21.36,9.73,11.49,-0.67,16.36,10.01,34.63,29.49,32.18,25.61,33.81,31.20,28.51,33.50,32.64,31.29,38.04,35.49,21.62,25.93,29.30,18.97,26.01,27.06,21.39,21.71,25.26,21.97,14.11,17.85,29.73,20.60,24.61,23.34,32.03,32.17,23.36,27.51,19.72,26.63,21.75,24.67,35.49,33.09,31.92,29.77,39.53,33.53,4.33,16.52,9.73,8.35,17.17,18.14,4.07,17.50,21.89,33.17,22.06,32.72


In [ ]:
rf_3_cls_all_zero.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,1.20,0.20,-2.59,-1.00,3.19,-1.20,-1.37,-5.60,-0.49,-1.76,-0.33,-1.43,-14.59,-7.21,-9.96,-14.79,-4.28,-8.16,1.70,2.71,1.92,0.34,-3.73,4.19,-0.70,-5.62,0.35,-2.81,-0.53,-1.14,-3.21,-0.43,-3.12,-2.26,-1.30,2.52,1.48,-0.10,1.18,-0.89,1.38,3.54,-3.79,-0.81,-2.87,-7.79,3.41,-0.97,0.49,2.63,1.97,0.74,-0.08,-0.49,-1.47,1.68,1.61,1.32,0.22,-2.20
pre_test,19.90,17.05,12.62,3.79,27.62,18.70,22.91,19.57,21.66,18.06,20.01,21.22,21.96,24.03,23.22,22.68,22.97,24.67,20.56,24.16,22.22,16.88,15.99,24.46,19.80,17.43,20.58,17.05,15.04,16.62,19.08,19.44,17.92,18.10,20.92,23.04,21.33,20.76,20.18,19.81,20.45,24.57,24.36,22.05,21.67,18.12,24.68,20.86,19.84,22.45,24.61,18.90,17.13,17.48,12.10,20.29,19.92,20.74,16.25,22.53
rec_test,1.20,0.20,-2.59,-1.00,3.19,-1.20,-1.37,-5.60,-0.49,-1.76,-0.33,-1.43,-14.59,-7.21,-9.96,-14.79,-4.28,-8.16,1.70,2.71,1.92,0.34,-3.73,4.19,-0.70,-5.62,0.35,-2.81,-0.53,-1.14,-3.21,-0.43,-3.12,-2.26,-1.30,2.52,1.48,-0.10,1.18,-0.89,1.38,3.54,-3.79,-0.81,-2.87,-7.79,3.41,-0.97,0.49,2.63,1.97,0.74,-0.08,-0.49,-1.47,1.68,1.61,1.32,0.22,-2.20
auc_test,27.15,21.37,17.80,20.89,25.32,24.05,27.50,24.75,25.90,23.79,28.41,26.34,27.63,28.12,28.82,30.62,30.61,30.60,23.26,22.55,21.28,22.43,22.21,23.27,19.62,19.67,22.47,19.63,20.88,21.08,21.74,18.94,20.90,21.91,24.09,25.25,20.95,22.60,21.97,21.44,22.08,24.95,28.43,26.18,26.06,27.30,30.25,27.10,19.15,21.34,19.72,18.51,21.40,23.39,21.02,22.20,23.59,26.26,23.54,26.19
f1_test,10.76,6.28,5.43,0.72,10.04,8.75,12.49,8.22,12.66,9.82,11.51,11.95,-0.79,7.28,4.29,-1.12,9.32,6.02,11.03,14.43,13.84,8.34,7.59,13.81,11.21,8.02,12.13,8.87,5.33,8.32,9.62,10.78,9.38,9.85,11.62,14.58,12.13,12.36,10.26,11.46,8.74,14.53,10.62,12.53,11.11,5.61,16.02,11.68,3.93,10.14,6.95,5.70,8.29,8.12,2.98,9.22,10.45,12.50,8.53,11.86
kappa_test,24.65,13.08,14.78,2.37,19.44,13.10,36.32,31.22,33.88,27.40,35.51,33.01,29.76,34.69,33.72,32.44,39.26,36.62,24.07,28.34,31.56,21.46,28.71,29.76,23.60,23.88,27.47,24.16,16.34,20.11,31.82,22.93,26.85,25.55,34.08,34.41,25.64,29.71,21.97,28.85,24.08,27.11,37.18,34.85,33.67,31.71,41.21,35.25,6.58,18.63,11.85,10.56,19.35,20.18,6.13,19.49,24.00,35.21,23.94,34.72


In [ ]:
rf_3_cls_all_rand.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_3_cls_all_rand.csv')
rf_3_cls_all_zero.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_rf_3_cls_all_zero.csv')

#### Бустинг 3 класса

In [ ]:
loggs = pd.read_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_3_cls_boost_all.csv', index_col=0)

In [ ]:
warnings.simplefilter('ignore')
boost_3_cls_all_rand, boost_3_cls_all_zero = get_effects_neutral(tg_train, tg_test, loggs)
warnings.simplefilter('default')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
boost_3_cls_all_rand.style.applymap(_color_red_or_green).format(precision=2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,33.17,32.52,31.11,27.56,26.88,34.89,30.86,29.59,28.84,27.57,28.28,29.74,33.79,30.93,31.59,29.74,30.55,30.93,27.23,27.80,25.12,25.18,24.40,25.75,23.02,21.86,25.32,21.84,23.12,22.17,25.71,24.30,21.90,23.50,26.34,22.39,25.75,22.54,24.38,23.60,22.17,26.21,30.36,26.41,28.97,27.80,27.61,30.04,24.68,26.09,24.60,24.15,25.47,28.35,26.40,26.51,26.27,27.18,28.18,25.41
pre_test,30.27,30.63,28.06,24.01,30.70,37.03,28.41,27.68,28.57,26.96,28.20,30.47,30.88,32.36,31.14,28.88,31.25,30.87,23.30,27.28,23.83,21.56,21.32,25.72,21.12,20.17,24.07,19.38,20.53,20.75,20.33,24.70,20.85,23.26,25.75,21.36,23.38,21.36,23.31,21.58,20.26,26.42,28.40,26.68,29.51,27.10,28.95,28.67,24.53,26.40,25.79,24.92,27.00,26.84,24.12,26.60,25.76,25.33,27.52,24.18
rec_test,33.17,32.52,31.11,27.56,26.88,34.89,30.86,29.59,28.84,27.57,28.28,29.74,33.79,30.93,31.59,29.74,30.55,30.93,27.23,27.80,25.12,25.18,24.40,25.75,23.02,21.86,25.32,21.84,23.12,22.17,25.71,24.30,21.90,23.50,26.34,22.39,25.75,22.54,24.38,23.60,22.17,26.21,30.36,26.41,28.97,27.80,27.61,30.04,24.68,26.09,24.60,24.15,25.47,28.35,26.40,26.51,26.27,27.18,28.18,25.41
auc_test,39.83,38.14,39.97,40.04,39.58,41.16,38.42,38.13,39.35,37.86,39.94,39.95,41.51,41.42,41.66,41.93,42.85,41.63,33.26,34.28,32.47,31.63,32.88,31.98,30.36,28.52,31.06,29.37,30.89,31.26,30.74,30.18,29.78,32.74,34.25,31.51,31.18,29.60,31.01,30.82,29.43,32.62,38.01,37.22,39.36,40.63,39.59,39.07,30.40,34.37,33.04,33.49,36.60,36.64,33.99,34.20,35.07,34.66,37.88,35.38
f1_test,31.82,31.58,29.77,25.97,28.61,36.37,29.69,28.66,28.73,27.32,28.24,30.16,32.42,31.27,31.31,29.10,30.82,30.87,25.24,27.09,23.88,23.20,22.46,24.23,21.60,20.86,24.42,20.60,21.56,20.83,23.40,24.50,21.45,23.43,25.94,21.78,24.11,21.84,23.62,22.56,21.08,25.88,29.41,26.49,29.23,27.29,28.37,29.43,21.54,25.34,23.97,23.62,24.42,26.27,25.14,26.22,25.74,26.20,27.67,24.73
kappa_test,62.41,63.21,61.88,59.12,58.51,66.69,58.18,58.59,58.69,57.58,62.99,62.24,64.11,64.39,65.20,63.98,68.69,65.50,46.31,50.33,44.92,46.44,46.56,46.29,38.68,39.70,46.92,40.24,45.58,40.63,43.99,48.01,43.58,47.83,53.88,42.85,43.99,42.25,45.83,43.48,42.69,48.48,57.97,54.89,60.28,61.06,60.71,62.07,36.69,47.10,44.68,45.67,49.34,51.11,49.16,52.86,52.37,55.36,59.42,48.18


In [ ]:
boost_3_cls_all_zero.style.applymap(_color_red_or_green).format(precision=2)

,AFLT_5_min_3_cls,AFLT_10_min_3_cls,AFLT_15_min_3_cls,AFLT_30_min_3_cls,AFLT_1_hour_3_cls,AFLT_1_day_3_cls,VTBR_5_min_3_cls,VTBR_10_min_3_cls,VTBR_15_min_3_cls,VTBR_30_min_3_cls,VTBR_1_hour_3_cls,VTBR_1_day_3_cls,GAZP_5_min_3_cls,GAZP_10_min_3_cls,GAZP_15_min_3_cls,GAZP_30_min_3_cls,GAZP_1_hour_3_cls,GAZP_1_day_3_cls,GMKN_5_min_3_cls,GMKN_10_min_3_cls,GMKN_15_min_3_cls,GMKN_30_min_3_cls,GMKN_1_hour_3_cls,GMKN_1_day_3_cls,LKOH_5_min_3_cls,LKOH_10_min_3_cls,LKOH_15_min_3_cls,LKOH_30_min_3_cls,LKOH_1_hour_3_cls,LKOH_1_day_3_cls,ROSN_5_min_3_cls,ROSN_10_min_3_cls,ROSN_15_min_3_cls,ROSN_30_min_3_cls,ROSN_1_hour_3_cls,ROSN_1_day_3_cls,NVTK_5_min_3_cls,NVTK_10_min_3_cls,NVTK_15_min_3_cls,NVTK_30_min_3_cls,NVTK_1_hour_3_cls,NVTK_1_day_3_cls,SBER_5_min_3_cls,SBER_10_min_3_cls,SBER_15_min_3_cls,SBER_30_min_3_cls,SBER_1_hour_3_cls,SBER_1_day_3_cls,TCSG_5_min_3_cls,TCSG_10_min_3_cls,TCSG_15_min_3_cls,TCSG_30_min_3_cls,TCSG_1_hour_3_cls,TCSG_1_day_3_cls,YNDX_5_min_3_cls,YNDX_10_min_3_cls,YNDX_15_min_3_cls,YNDX_30_min_3_cls,YNDX_1_hour_3_cls,YNDX_1_day_3_cls
acc_test,15.74,15.94,13.55,9.56,15.74,23.51,11.69,11.03,12.68,11.09,12.18,14.60,13.99,14.54,14.41,11.56,14.69,14.44,8.82,12.78,9.39,7.47,7.58,10.75,6.50,5.62,9.57,4.74,6.23,6.41,5.21,9.11,5.56,7.47,11.02,7.03,8.66,6.99,8.96,6.99,5.91,12.60,11.90,10.12,13.31,10.28,13.04,12.99,8.87,12.07,11.33,10.51,11.25,11.58,9.16,11.72,10.84,10.48,12.75,9.08
pre_test,37.14,37.36,34.13,28.68,38.44,47.33,34.54,33.28,34.60,32.38,33.04,36.55,36.42,38.33,36.54,33.44,35.99,36.12,29.95,35.10,30.73,26.96,26.37,32.30,28.54,26.98,30.92,25.47,25.21,26.65,25.85,31.64,26.96,29.60,31.62,27.95,30.31,28.24,30.15,28.09,25.97,34.98,34.62,32.66,35.71,31.79,34.75,33.88,32.11,34.44,33.87,32.20,32.82,32.92,30.08,32.88,31.57,30.51,32.85,30.66
rec_test,15.74,15.94,13.55,9.56,15.74,23.51,11.69,11.03,12.68,11.09,12.18,14.60,13.99,14.54,14.41,11.56,14.69,14.44,8.82,12.78,9.39,7.47,7.58,10.75,6.50,5.62,9.57,4.74,6.23,6.41,5.21,9.11,5.56,7.47,11.02,7.03,8.66,6.99,8.96,6.99,5.91,12.60,11.90,10.12,13.31,10.28,13.04,12.99,8.87,12.07,11.33,10.51,11.25,11.58,9.16,11.72,10.84,10.48,12.75,9.08
auc_test,41.64,39.77,41.78,41.92,41.09,42.73,39.37,39.03,40.16,38.67,40.81,40.80,42.12,41.99,42.23,42.57,43.46,42.22,34.58,35.52,33.72,32.85,34.26,33.14,31.46,29.62,32.15,30.52,31.97,32.36,31.93,31.22,30.96,33.90,35.30,32.58,32.30,30.79,32.11,31.96,30.56,33.80,38.90,38.07,40.25,41.54,40.41,39.92,31.44,35.31,34.00,34.53,37.62,37.72,35.00,35.23,36.01,35.75,38.94,36.38
f1_test,28.35,28.54,25.55,20.37,28.84,38.13,25.04,23.93,25.70,23.61,24.24,27.68,27.05,28.19,27.34,23.86,26.96,27.13,21.29,25.90,21.62,18.60,18.05,22.10,19.39,18.27,22.11,16.95,16.93,17.75,17.34,22.62,18.25,20.60,23.06,19.37,21.22,19.62,21.40,19.52,17.61,25.55,25.26,23.35,26.57,22.39,25.87,25.25,19.84,24.69,23.69,22.47,22.09,22.67,21.42,23.98,22.85,22.09,24.26,21.93
kappa_test,65.86,66.42,65.29,62.57,61.84,69.92,59.94,60.32,60.33,59.20,64.75,63.99,65.27,65.56,66.35,65.22,69.92,66.69,48.83,52.84,47.44,48.77,49.24,48.68,40.86,41.90,49.13,42.49,47.70,42.85,46.16,50.14,45.94,50.18,56.01,45.01,46.20,44.70,48.06,45.74,44.95,50.88,59.67,56.60,62.10,62.82,62.43,63.77,38.80,49.07,46.71,47.83,51.48,53.23,51.13,54.99,54.30,57.51,61.55,50.21


In [ ]:
boost_3_cls_all_rand.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_3_cls_all_rand.csv')
boost_3_cls_all_zero.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/results_boost_3_cls_all_zero.csv')

## Оценка резульатов

In [ ]:
results = os.listdir('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs')
results = [f for f in results if 'results' in f][1:]

In [ ]:
results

['results_rf_2_cls.csv',
 'results_rf_2_cls_embs.csv',
 'results_boost_2_cls_embs.csv',
 'results_rf_2_cls_all.csv',
 'results_boost_2_cls_all.csv',
 'results_boost_2_cls.csv',
 'results_rf_3_cls_rand.csv',
 'results_rf_3_cls_zero.csv',
 'results_boost_3_cls_rand.csv',
 'results_boost_3_cls_zero.csv',
 'results_rf_3_cls_emb_rand.csv',
 'results_rf_3_cls_emb_zero.csv',
 'results_boost_3_cls_emb_rand.csv',
 'results_boost_3_cls_emb_zero.csv',
 'results_rf_3_cls_all_rand.csv',
 'results_rf_3_cls_all_zero.csv',
 'results_boost_3_cls_all_rand.csv',
 'results_boost_3_cls_all_zero.csv']

In [ ]:
def get_mean_res(df):
  res_comp = pd.DataFrame(index=df.index)
  for company in companies:
    df_tmp = df[[col for col in df.columns if company in col]].copy()
    res_comp[company] = df_tmp.mean(axis=1)
  return res_comp

In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[0]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,15.08,14.07,14.64,11.13,9.69,9.92,10.55,13.35,9.20,11.55
pre_test,15.86,14.09,16.24,12.90,12.79,13.66,13.81,14.62,10.75,14.05
rec_test,12.47,12.93,8.74,3.63,-2.04,-3.05,-0.07,8.70,3.20,1.09
auc_test,20.00,17.80,19.20,14.91,12.47,13.68,14.70,17.56,13.28,15.12
f1_test,14.15,13.54,12.34,7.96,4.53,4.19,6.12,11.41,6.24,7.11
kappa_test,30.08,28.14,29.23,21.87,19.42,19.76,21.36,26.55,18.27,23.54


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[1]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,12.92,13.10,14.29,9.47,9.05,9.04,9.72,12.43,7.78,9.24
pre_test,13.35,12.77,16.13,11.51,11.48,12.29,13.57,14.05,8.06,12.23
rec_test,11.01,13.10,7.66,-0.04,-1.73,-2.86,-2.54,6.16,7.02,-3.14
auc_test,16.02,16.82,17.05,11.90,11.74,11.95,13.17,15.71,11.85,12.28
f1_test,12.28,12.98,11.70,5.28,4.34,3.79,4.51,9.85,6.76,3.64
kappa_test,25.77,26.14,28.55,18.65,18.24,18.00,19.94,24.80,15.71,19.02


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[3]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,10.84,13.17,13.74,9.17,8.62,9.08,9.74,12.68,6.37,9.36
pre_test,11.35,12.85,16.69,10.67,11.07,12.04,12.99,14.03,6.85,11.59
rec_test,8.94,13.21,3.90,1.19,-3.00,-2.79,-1.23,7.08,2.74,-0.95
auc_test,14.74,16.25,17.33,12.56,10.57,11.98,12.74,15.59,10.15,12.47
f1_test,10.12,13.08,9.65,5.62,3.41,3.83,5.14,10.26,4.40,4.71
kappa_test,21.83,26.25,27.64,18.05,17.30,18.13,19.80,25.16,12.73,19.24


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[5]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,13.70,12.98,13.58,8.83,8.46,8.42,8.97,12.30,8.63,10.02
pre_test,14.26,13.03,13.63,9.08,10.00,9.50,10.59,12.88,9.10,11.03
rec_test,10.88,11.50,11.79,6.65,0.22,2.36,2.55,9.02,6.24,3.16
auc_test,18.93,17.92,19.71,13.52,12.26,13.52,13.92,17.15,11.56,14.29
f1_test,12.72,12.35,12.77,8.02,4.97,5.90,6.31,11.01,7.73,7.12
kappa_test,27.12,25.98,27.15,17.36,16.95,16.93,18.02,24.52,17.25,20.25


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[2]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,11.87,11.97,13.54,6.94,8.78,9.05,7.60,11.95,6.78,8.02
pre_test,12.79,11.46,13.45,7.21,9.65,9.72,8.37,12.66,6.24,8.48
rec_test,8.03,12.53,12.30,3.42,3.17,4.60,2.37,8.36,10.19,3.25
auc_test,16.01,16.61,18.98,10.59,11.89,13.36,11.14,16.27,10.41,11.97
f1_test,10.45,12.09,12.95,5.42,6.44,7.25,5.37,10.53,8.12,5.99
kappa_test,23.96,23.81,27.02,13.63,17.66,18.11,15.09,23.76,13.80,16.08


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[4]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,12.81,12.72,13.76,7.58,7.75,9.71,9.01,11.96,8.00,8.94
pre_test,13.53,12.57,13.59,8.13,8.93,10.49,10.53,12.29,7.65,9.58
rec_test,9.56,11.80,12.87,3.20,0.35,5.17,2.70,9.13,9.33,3.43
auc_test,17.96,17.58,19.62,11.97,11.35,14.11,12.60,16.70,11.38,13.24
f1_test,11.66,12.26,13.28,5.76,4.59,7.91,6.43,10.77,8.54,6.53
kappa_test,25.27,25.47,27.50,14.91,15.60,19.45,17.91,23.77,16.11,17.96


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[6]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,14.61,5.09,-2.49,13.20,2.12,0.84,8.32,-1.66,13.10,10.80
pre_test,19.74,20.91,21.75,16.93,15.97,16.51,16.98,20.96,19.32,18.49
rec_test,14.61,5.09,-2.49,13.20,2.12,0.84,8.32,-1.66,13.10,10.80
auc_test,31.17,31.97,34.84,27.27,25.75,26.41,26.12,32.05,28.17,28.64
f1_test,15.76,5.64,-4.17,13.80,2.16,0.99,9.09,-2.93,14.60,12.11
kappa_test,38.68,32.97,28.68,33.88,26.47,26.45,30.72,28.75,36.70,35.83


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[7]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,-0.80,-11.83,-19.81,-3.28,-14.25,-15.65,-7.43,-18.23,-1.50,-5.19
pre_test,26.81,26.61,26.98,23.33,22.27,22.74,23.88,26.63,26.47,24.32
rec_test,-0.80,-11.83,-19.81,-3.28,-14.25,-15.65,-7.43,-18.23,-1.50,-5.19
auc_test,32.88,32.85,35.44,28.53,26.86,27.52,27.27,32.92,29.20,29.64
f1_test,13.33,1.89,-8.39,10.69,-0.91,-2.24,6.73,-6.51,12.99,8.90
kappa_test,42.02,34.70,29.86,36.37,28.68,28.66,33.01,30.47,38.81,37.86


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[8]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,28.47,25.61,28.30,22.59,17.98,19.61,20.75,25.69,24.42,23.00
pre_test,29.47,27.50,29.83,23.14,20.59,21.81,22.40,27.83,25.29,24.69
rec_test,28.47,25.61,28.30,22.59,17.98,19.61,20.75,25.69,24.42,23.00
auc_test,40.62,39.04,41.42,33.45,30.26,31.97,31.61,39.07,34.92,35.49
f1_test,28.78,25.85,28.39,22.64,18.60,20.16,21.32,26.07,24.97,23.48
kappa_test,60.12,56.08,61.45,47.33,41.53,44.04,44.68,56.96,50.62,51.31


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[9]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,13.08,8.68,10.98,6.15,1.61,3.11,5.02,9.13,9.82,7.00
pre_test,36.56,33.20,35.06,29.53,26.88,28.02,29.32,33.52,32.46,30.50
rec_test,13.08,8.68,10.98,6.15,1.61,3.11,5.02,9.13,9.82,7.00
auc_test,42.34,39.93,42.01,34.70,31.37,33.07,32.77,39.95,35.97,36.49
f1_test,26.38,22.10,24.17,19.55,15.53,16.92,18.97,22.50,23.37,20.26
kappa_test,63.51,57.82,62.62,49.80,43.73,46.22,47.01,58.70,52.75,53.31


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[10]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,14.82,17.42,13.17,18.22,16.15,14.74,15.82,15.44,15.48,15.15
pre_test,9.79,15.76,18.68,14.54,11.44,11.37,12.18,14.87,11.87,12.46
rec_test,14.82,17.42,13.17,18.22,16.15,14.74,15.82,15.44,15.48,15.15
auc_test,21.16,26.03,29.26,20.97,18.28,19.59,19.86,25.58,19.32,21.87
f1_test,7.48,16.43,13.98,12.61,11.47,12.56,10.95,14.94,7.36,11.88
kappa_test,8.04,32.27,37.34,19.37,16.80,21.38,16.47,30.48,9.36,21.22


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[11]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,-0.60,0.46,-4.18,1.79,-0.25,-1.75,0.10,-1.14,0.88,-0.83
pre_test,16.84,21.43,23.90,20.96,17.71,17.59,19.09,20.54,19.03,18.29
rec_test,-0.60,0.46,-4.18,1.79,-0.25,-1.75,0.10,-1.14,0.88,-0.83
auc_test,22.84,26.87,29.84,22.25,19.36,20.69,21.03,26.44,20.35,22.88
f1_test,5.06,12.64,9.75,9.54,8.37,9.33,8.61,11.36,5.75,8.68
kappa_test,11.36,33.94,38.49,21.91,18.96,23.57,18.81,32.20,11.48,23.25


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[12]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,30.43,29.30,31.04,25.54,22.74,23.83,24.12,28.06,25.51,26.50
pre_test,29.42,28.54,30.80,23.46,20.93,22.46,22.75,27.80,25.71,25.42
rec_test,30.43,29.30,31.04,25.54,22.74,23.83,24.12,28.06,25.51,26.50
auc_test,38.42,37.93,39.58,31.35,28.86,29.88,29.37,37.52,33.75,34.35
f1_test,29.92,28.96,30.78,23.97,21.63,23.18,23.20,27.92,24.22,25.80
kappa_test,59.99,60.01,65.02,45.97,42.27,46.13,44.53,58.65,45.76,52.64


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[13]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,15.04,12.37,13.72,9.09,6.35,7.35,8.42,11.50,10.89,10.51
pre_test,36.47,34.23,36.03,29.86,27.20,28.68,29.66,33.47,32.85,31.24
rec_test,15.04,12.37,13.72,9.09,6.35,7.35,8.42,11.50,10.89,10.51
auc_test,40.09,38.80,40.18,32.62,29.95,30.99,30.52,38.37,34.78,35.35
f1_test,27.51,25.20,26.57,20.87,18.53,19.96,20.87,24.34,22.59,22.60
kappa_test,63.30,61.75,66.20,48.47,44.42,48.34,46.84,60.35,47.84,54.66


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[14]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,15.42,15.10,7.50,17.65,14.62,15.19,16.85,14.42,15.48,16.18
pre_test,9.62,14.87,18.02,14.31,11.45,13.53,14.29,16.26,12.90,12.82
rec_test,15.42,15.10,7.50,17.65,14.62,15.19,16.85,14.42,15.48,16.18
auc_test,21.15,25.24,28.81,21.23,19.44,21.04,21.20,26.67,19.53,22.80
f1_test,9.47,14.86,8.39,14.60,12.05,14.20,13.94,14.83,8.79,12.46
kappa_test,11.38,31.15,33.24,24.81,20.38,27.08,23.94,33.89,12.37,21.90


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[15]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,-0.03,-1.83,-9.83,1.19,-1.74,-1.30,1.10,-2.14,0.88,0.19
pre_test,16.61,20.57,23.26,20.71,17.75,19.75,21.18,21.96,20.07,18.64
rec_test,-0.03,-1.83,-9.83,1.19,-1.74,-1.30,1.10,-2.14,0.88,0.19
auc_test,22.76,26.11,29.40,22.50,20.56,22.14,22.33,27.55,20.59,23.80
f1_test,7.00,11.11,4.17,11.51,8.98,10.97,11.58,11.26,7.19,9.26
kappa_test,14.57,32.89,34.41,27.32,22.59,29.27,26.23,35.65,14.53,23.92


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[16]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,31.02,29.15,31.25,25.91,22.89,24.02,24.11,28.53,25.56,26.66
pre_test,30.12,28.38,30.90,23.83,21.00,22.71,22.72,28.22,25.91,25.59
rec_test,31.02,29.15,31.25,25.91,22.89,24.02,24.11,28.53,25.56,26.66
auc_test,39.79,38.94,41.83,32.75,30.24,31.53,30.78,38.98,34.09,35.20
f1_test,30.69,28.80,30.96,24.35,21.64,23.42,23.18,28.37,24.19,25.95
kappa_test,61.97,59.71,65.31,46.81,41.96,46.69,44.45,59.50,45.76,52.89


In [ ]:
get_mean_res(pd.read_csv(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{results[17]}', index_col=0)).style.applymap(_color_red_or_green).format(precision=2)

,AFLT,VTBR,GAZP,GMKN,LKOH,ROSN,NVTK,SBER,TCSG,YNDX
acc_test,15.67,12.21,13.94,9.46,6.51,7.57,8.35,11.94,10.94,10.67
pre_test,37.18,34.06,36.14,30.24,27.30,28.94,29.62,33.90,33.06,31.43
rec_test,15.67,12.21,13.94,9.46,6.51,7.57,8.35,11.94,10.94,10.67
auc_test,41.49,39.81,42.43,34.01,31.35,32.65,31.92,39.85,35.10,36.22
f1_test,28.30,25.03,26.75,21.26,18.57,20.21,20.82,24.78,22.57,22.75
kappa_test,65.32,61.42,66.50,49.30,44.16,48.91,46.76,61.23,47.85,54.95


## Перезапуск обученных моделей

In [ ]:
num_cols = ['5_min', '10_min', '15_min', '30_min', '1_hour', '1_day']
target_cols = [col + '_2_cls' for col in num_cols]
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, ngram_range=(1, 3), max_features=1000)
tfidf.fit(tg_train.message_lemm.values)
emb_train = embs[:len(tg_train), :]
emb_test = embs[len(tg_train):, :]
for company in companies:
  n_jobs=-1
  print(company)
  os.makedirs(f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{company}', exist_ok=True)

  comp_train = tg_train[tg_train[company] == True].copy()
  comp_test = tg_test[tg_test[company] == True].copy()

  emb_features_train = emb_train[tg_train[company] == True]
  emb_features_test = emb_test[tg_test[company] == True]

  text_features_train = tfidf.transform(comp_train['message_lemm'].values)
  text_features_test = tfidf.transform(comp_test['message_lemm'].values)

  text_features_train = np.asarray(text_features_train.todense())
  text_features_test = np.asarray(text_features_test.todense())

  text_features_train = np.concatenate([text_features_train, emb_features_train], axis=1)
  text_features_test = np.concatenate([text_features_test, emb_features_test], axis=1)

  for t, n in zip(target_cols, num_cols):
    print(t)
    y_train = comp_train[t].values.astype(int)
    y_test = comp_test[t].values.astype(int)
    float_cols = sorted([col for col in tg_train.columns if re.search(f'^{n}_lag', col)])

    num_features_train = comp_train[float_cols]
    num_features_test = comp_test[float_cols]

    features_train = np.concatenate([text_features_train, num_features_train], axis=1)
    features_test = np.concatenate([text_features_test, num_features_test], axis=1)

    try:
      filename = f'/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/{company}/{t}_2_cls_forest_all.sav'
      gc = pickle.load(open(filename, 'rb'))
      preds_test = gc.predict(features_test)
      preds_proba_test = gc.predict_proba(features_test)

      acc_test = accuracy_score(y_test, preds_test)
      pre_test = precision_score(y_test, preds_test)
      rec_test = recall_score(y_test, preds_test)
      auc_test = roc_auc_score(y_test, preds_proba_test[:,1])
      f1_test = f1_score(y_test, preds_test)
      kappa_test = cohen_kappa_score(y_test, preds_test)

      df[f'{company}_{t}'] = [acc_test, pre_test, rec_test, auc_test, f1_test, kappa_test]
    except FileNotFoundError:
      print(f'Надо посчитать {company} {t}')

    df.to_csv('/content/drive/MyDrive/Нейросети для исследований научка/tg_loggs/loggs_2_cls_forest_all.csv')